In [13]:
import os
import logging
import concurrent.futures
from typing import List, Dict, Any, Callable, Union, Optional
from pydantic import BaseModel

from haystack import Pipeline, Document, component
from haystack.utils import Secret
from haystack.components.converters import PyPDFToDocument
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.embedders import OpenAIDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from haystack_integrations.document_stores.pinecone import PineconeDocumentStore
from haystack.components.generators import OpenAIGenerator as BaseOpenAIGenerator
from haystack.components.builders import PromptBuilder
from haystack.dataclasses import ChatMessage, StreamingChunk
from openai import OpenAI, Stream
from openai.types.chat import ChatCompletion, ChatCompletionChunk
from haystack.components.generators.openai_utils import _convert_message_to_openai_format
import concurrent.futures
from typing import List, Dict
from tqdm import tqdm
from haystack import component
from haystack.dataclasses import Document
from anthropic import Anthropic
from dotenv import load_dotenv

In [14]:
load_dotenv()

import numpy as np
from tqdm import tqdm


In [15]:
# Logging config
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [16]:
text = """
St. Jeemps den 9e\n19 Maert\n1690\nEergisteren middach sijn wij hier\ngearriveert, en hebben soo commodieu-\nsen, en gemackelijcken passasie gehad,\nof wij van Haeg naer Uijtrecht\nmet een trekschuijt voeren, uijt-\ngesondert, dat vermits ons te lang\nvertoeven, met moijte aent oor-\nloch schip geraeckte gelijck int bree-\nde aen Vrou van Ginkel schrijve.\nDe koninck en koningin hebbe\nick in gesontheijt gevonden en ben\nvan selve met goedertierentheijt\nontfangen, U.H.ed. brief hebbe aen\nden koninck eijgenhandich overgeheven\n\nDoch noch geene occasie gehad, den\nkoninck in 't particulier te spreec-\nken, het welcke echter verhoope\ndat morgen of overmorgen sal kun-\nnen doen. De ijersche reijse soo wel\nvoor den koninck als ons staet noch\nvast, ten waere dat bij 't sitten\nvan 't nieuwe parlement eenige ver-\nandering voor quam, het welcke men\nechter niet gelooft, maer dat se goedt\nwesen sullen, altoos de verkiesingen lae-\nten sich tot noch toe wel aen sien\nen werden de presbiteriaenen het mees-\nte uijtgeknijpt, in Ijerlandt hebben\nonse troepen eenige victorie bevochten\ngelijck U. H. Ed. uijt het nevengaen-\nde sal gelieven te sien. Ick hoop dat\nmet de laeste post sal seggen kun-\nnen wanneer dat mijne reijse aen-\nvanck nemen sal, U. H. Ed. missive\nvan den 10de hebbe gisteren naer middach\nontfangen, en met vreuchde daer\nuijt gesien, dat de saecken met Am-\nsterdam sijn bijgelegt, dat soo wel\nhier, als in onsen Staet veel goedts aen\nden koninck wil doen, den Marquies\nvan Mompoullan, en den Graef van\nFlodorp is met sijn mon Ametje\nhier, en seijdt den Marquies van\nMompoullan dat hij voor volon-\ntaire met naer Ijerlandt gaen wil,\n\nHet is een geluck dat den staedt van\nsoo  grooten quantiteijt goede gene-\naels is voorsien, dat se der  omvoor\nvolontaires te gaen dienen, missen\nkunnen, doch als het eene schip af-\nvaert soo leijt het aender neder aen-\nboort, maer van een levendich exem-\npel inde Commandeur van Utrecht ge-\nsien werdt, ick ben Broer Appelaer\ngeobligeert dat mijn met de paerden\ninde courantes heeft geset, vele vrien-\nden hebben hier naer UH.Ed. gesontheijt\ngevraecht, en presenteren alle haeren\ndienst, met naemen de heeren\nvan Zithers, Hop, secretaris de Wilt\nen vele andere ick blijve met haest\n\n\nSt. Jeemps den 25 Maert 1690\nIck hebbe d'eere gehad t'ontfangen twee brieven\nvan U.H.Ed: beijde vande 19e deses doch den eenen\nmet U.H.Ed.eijgen handt geschreven en ben\nU.H.Ed. onderdanich bedankend voorden precan-\ntien, en sorge die U.H.Ed: voor mijn en tot\nconservatie van rust en vreede inde mijne fami-\nlie,door sijne dispositien sal nemen willen\nhet doet mijn leet, dat wij vermits d'opinia,\nter heijt vande vrouwe van Ginckel dit met\ngesamentlijck hebben kunnen doen wil-\nde ickhe hoopen dat alles noch ten besten\nuijt vallen sal en dat ick en onse kinderen\nons ten uijt eijnde onses levens soo danich\ngedragen sullen dat en U.H.Ed: ende  V. van A\n\nAltoos een volle benoegen aen ons sullen\nhouden en geve godts, dat U.H.Ed: mede met\ngesontheijt op Amerongen konnen mach\nen noch lange jaeren door brengen Hoe onse\nijersche campagne noch af sal loopen leert\nden tijdt en hoet tussen Graef Meijnart\nvan Schomberch en mijn sal worden in\ngeschickt, ick hoope dat den koninck een\nmiddel uijt vinden sal, ick deselve met\nbenoegen, en reputatie doen kan soo niet\nmoet patientie hebben, en stil t'huijs\nkomen ick sal ondertussen bidden dat bij de\naen of sterven mijn vrou en kinderen altoos\nhaer gunst mach blijven geconserveert,\nen de swackheijt van V. van G: wat\nmach werden toe gegeven. Uijt mijne twee\nlaetste heeft U.H.ed gesien het antwoort dat\nde koning mijn gegeven heeft opt geene\nhem van U.H.Ed: wegen hebbe voor gedra-.\ngen en is niet te twijfelen of U.H.Ed.\nsal de selfde Charater vande Heere Haenen\nkrijgen soo daer op gelieft t'insteren.\ndat naer mijn oordeel uijt veele in-\nsichten U.H.Ed: avantasieus  is, en waerom\nte meer den konink daer over int bree-\nde onderhouden hebbe, de Graeven van\nSolms, Poortland, en Schonberch, hebbe\neen moeijelijke reijse gehad, waeren ver-\nzeijlt, en vonden sich voor Doevren (Dover)\nin plaets van voorde Revier van Londen\nsoo dat se een ganschen dach lange\n\nop zee sijn geweest danse gemeijnt\nhadden, en van Doevren tot aende Revier\nop laveren mosten met sonder perijc-\nkel en see sieck sijn d'aenspraeck\nvande Koninck aent parlement sal appa-\nrent met de gase...... publieck sijn\nwat nu bij't parlement geresolveert\nsal werden staet men alle daegen te\nhooren, het wil seer opt gelt geven\naen komen dat noch mancqueert en\noorsaeck is, de marsch naer IJerlandt\nniet seer verhaest werdt, men laet\nniet even groote imeparatien te maek-\nken, en werden onse equipages dagelijcks\ngrooter ick blijve etc..met grooten\nheast, bidde daerom gunste dat aen V. van de A\nniet schrijve.\nSt.Jeemps den\n18/28 Maert\n1690\nT'sedert UHE laetste missive vande 7/17 sijn\nder geene brieven uijt Hollandt gekomen\nsoo dat hier a drie posten mancqueren\napparent soo sijn alle parket booten\naen dese zijde, en men verwacht het\noverkomen van Graef van Poortlandt\nalle daegen ick hebbe dese morgen\noccasie gehad aende Konink over het\nbewuste te spreecken die seijde dat\nhem van al dat vruile  en valsch uijt\nstroijsel, niets nae ter ooren gekomen.\ndat deselve van UHE conduiten was\nvoldaen ,  en dat tot een teken aen\nUHE: verscheijde brieven van danck,\nsegging geschreven hadden dat hem\nwel van eenich misverstandt tussen\n\nDe prins van Waldijck en UHE: nae\ngeseijt, doch vraechde mijn vrede\nwaer het vast was, en als aldus selve\nalles verhaelde wat daer van wiste\nen hoet sich tot mijn vertreck toe\nhadde te gedraegen, geliefde sijne majestijt,\nden raet pensionaris noch niet was\ngeschiet, waerop ick repliceerde dat ter\napparent geene instantien het toe neemen\ngevaer, maer dat UHE: voor en\nnae den prinse van Waldijck sach\nnaermede dit disccours werde afgebrooc-\nken, en als ick verder van Deense\ncommissie meijnde te spreecken\nsoo vinde geintrumpeeert.\nHet sal in allen geval noch te passe\nkomen, wanneer wij hoorden dat U.H.Ed\ndeselve heeft aengenomen wanneer onse\nRuijterije marscheren sal en is noch niet-\nvast gestelt, Eenige Regimenten te voet\nbeginnen vast naer de havens te defile-\nren .Het Parlement staet overmorgen\nhaere zessien te beginnen en gelijck\nd'eerste twee a drie daegen met het\nmaecken van een spreecker, en dier gelijc-\nke saecken door gaen sal men met de\npost van heden over acht daegen\neerst recht eenich niuws en wat con-\ntenantie sij houden schrijven kunnen\nDe koninck heeft gisteren eene groote\nverandering in sijn Tresaurije  gemaeckt\nen .sijnder van vier Commissarissen uijt\nde welcke deselve bestont drie uijt\ngevallen als Milord Mordant milord\nGodolphijn en Heer Harrij Hoijpel den\n\nEersten is bij UHE bekent, die seer\nt'onreeden sijn sal. De twee soons\nvan den Hartoch van Bolton sijn mede\nint parlement, contre tout et Maree\nen heeft de Koninck gisteren aende jongste\nde schoon soon van Mompoullan in een\nscharge van twintich duijsent gulden s'jaers\ngegeven, men moet nu sien hoe se sich\nkomporteren sullen, ick sie dat se al\nveele leden vant oude parlement weder\ngeeligeert sijn doch t'is te hoopen dat\nde nieuwe sullen boven slaen t'welck\nmen sien sal, soo se haer begin\nmaecken met gelt te geven gelijck\nmen vertrouwt, ick blijve met offre\nvan mijn dienst aen Vrou van Amer-\nongen, etc ick sende aen den commies van\nHeeteren, de origineele verklaringe vande\ntresorier vander G.S van't gelt dat den Graef\nvan Solms, en ick uijt d'engelse Betalinge wese\nSt. Jeemps den 19e/29e maert 1690\nIck hebbe nog geene occasie gehad den\nkoninck te kunnen spreecken over het\ngeene bij mijne voorige hebbe gemelt\nen dat echter versoeke dat gemenageert\nmach werden men heeft hier ondertu-\nsen U.H.Ed: vertreck gedebiteert, en dat de\nselve de commissie aengenomen heeft\ndie hoope dat U.H.Ed met gesontheijt\nen contentement volherden sal om-\nt'openen van mijine brieven heeft de commies\nde Wilde mijn veele excuses gemaeckt en\nseijdt het bij abuijs en precipitantie\nis geschiet, dat mede al somwijlen\nsoude wesen kunnen, De Konninck\nheeft aende morgen tot den avont seer\n\nveel te doen, en men kan hem beswaerlijk\nergens over te spreecken komen aanstaende\ndonderdach sal het nieuwe parlement\nsitten en wil van haere conduiten\nveel dependeren ondertussen soo isser\nwat schaersheijt van gelt, uijt oorsaecke\ndat niemant op de laeste gecomen heeren\nde penningen gelt schieter wil, voor-\ndat se de continantie van't aenstaende\nparlement sien, en raecken de troepen\nten achteren, ick geloove dat U.H.Ed\nmet de voorige post de groote tijding uijt\nSwitserlandt geschreven hebbe, van dat de\nCantons van Barn en Zurich aen onse\nkoninck het nemen van 4000 man heb-\nben toe gestaen, en alle haere officieren\nhebben inde boetes geslaegen die het\nvoorleden jaer tegens het rijck hebben\ngedient, den  heer van s'Gravemoer\nis gisteren uijt IJerlandt gearriveert\ndick en vet, en seijdt dat de siecktens\ndaer ten eenenmael cesseren. De Deenen\nsijn al over en sal den Hartoch van\nSchonberch daer nu neder bij de 19000\nman bij den anderen hebben, van't franse\nsecours werdt divers gesproocken, doch\nse kunnen't senden,wanneer se willen\nsoo lange als onse vlooten niet in zee\nsijn. De koningin van Spaenjen is eijn-\ndelijck, en ten laetsten vertrocken, en\nsal apparent bij dit schoone weder\nnu al over sijn mijn verlangt dat\n\nDen Graef van Portland met Conte\nMenaert(Mijnhard) van Schonberch over kompt\nen wat Emploij hij hebben sal waer-\nmede verblijve\nSt jeemps den 21/31 maert\n1690\nick hebbe occasie gehad gisteren met den\nkonink van flandrich te spreeken\nde commissie van U.H. Ed naer Denemar-\nken en geseijt U.H.Ed wel voorgenom-\nen hadde niet meer buijten S'lants\nte gean, doch bij aktien het sijne\nMajesteit ernstige begeeve was en dat\ndeselve daer mede sonderlinge dienst\nkonde doen soo soude U.H.Ed licht daer-\ntoe resolveren waerop de Konink\ngeliefde t'antwoorden dat hem was\nbericht U.H.Ed tot dese commissie niet\nvoorgeslagen en dat sijn Majt. aenge-\nnaem soude sijn U.H.Ed deselve aennam\n\nbij aldien het sijne gesontheijt\ntoe liedt, te meer alsoo men bekom-\nmeringe kreech, als of deenmercken\nsich van Vrankrijck weder liedt\naensoecken, en als ick sijne majesteit\nverder sprak, over de character van\nambassadeur of envoijé, en de dachgel-\nden, soo seijde sijn Majesteit dat U. H. Ed.\ngelijck hadde, dat in geen minder qualiteit\nals den Heere van Haeren naer Sweeden\nwilde gaen, en dat de dachgelden voor\nalle drie aguael mosten sijn, soo se envo-\nijees waeren. Dit is met U. H. Ed. parmis-\nsie antwoort op dat point, ick salder\nalleen bij voegen dat den Envoijé\nzente?, suspect en voor frans gesint\ngehouden werdt, de koninck geliefde\nmijn oock te seggen, dat aen deselve\naengenaem wesen soude, U. H. Ed. met\nsijnen Envoijé correspondeerde, en\ndat hem geen oneffen man soude vinden,\nden Heere Hop meijndt echter dat\nhij niet van die soliditeijt en is, en\nmet voorsichtichheijt moet gehandelt\nwerden, mijn daer uijt sijn selven\nafspreekende, ick hebbe oock aen\nsijne Majesteit gevraecht, over U. H. Ed.\nde continuatie van U. H. Ed. commissie\ninden Raet, die seijde dat hem aenge-\nnaem was, dat U. H. Ed. daer in continu-\neerde, en dat het tussen de Ridderschap\nen de Stadt van Utrecht wierde\ningeschikt, ick vertrouwe dat\n\nSijn Majesteit daer over oock wel sal\nschrijven willen, maer de vraech is,\noft niet noch te vroech sij. Wat\nde gedachten vande heer van Wellandt\naenbelangt daer toe sie meijnich\napparentie, want men doet groote\ndevoinen voor Hulft van Brussel,\nen het intrim is bij provisie al\nop een jode geconfereert die men\nseijdt seer habiel te sijn. Het par-\nlement heeft gisteren sijn eerste sessie\ngehad, en het Legerhuijs heeft haer\nspreecker genoempt. Heden heeft den\nKonink sijne propositie int parlement\ngedaen doch se is noch niet uijt geko-\nmen, ick sal se U. H. Ed. met de laeste\npost kunnen toe senden, de Graef\nvan Solms, Poortlandt, en Schom-\nberch werden alle uur verwacht,\nen ben ick seer verlangende wat em-\nploij men Schomberch geven sal.\nMen debiteert hier dat het franse\nsecours al op d'ijersche kusten is\ngesien, de Deenen sijn over, en\nettelijcke Regimenten te voet mar-\ncheren mede doch onse Marsch\nis noch niet gearresteert, men\nwacht naert gelt en den uijtslach vant\nparlement dat men gelooft wel sal sijn.\nIck blijve met d'offres van mijn gehoor-\nsame dienst aen Vrou van Amerongen\netcetera\n\nDen commissaris van der Esch is sollici-\nterende naer't penningmeesterschap\nvan Schielandt, vacant geworden door\nde doot van den Heer s'Gravemoers\nswaeger, en heeft den secretaris\nBeaumont voor sijn patroon, die\nneffens den Heer van Achtienhoven\nen d'andere Heemvaeder van Delflant\ndaer van d'Electie heeft, en soude\nIck gedienstich bidden, dat bij aldier\nVan der Es op de nominatie kompt,\nU. H. Ed. hem bij den  Heer van Actien\nover tot d'electie met ernst gelief-\nde te recommanderen.\nSt Jeemps den 1en april 1690\nMen seijdt dat ter desen dach Hollant-\nse brieven aengekomen sijn, doch ick\nhebbe der tot noch toe geene ontfan-\ngen, soo dat niet weete of U. H. Ed al\nuijt den Haeg vertrocken is, of dat\nsich noch daer bevindt, ick hoope\ndat alle mijne brieven wel overkomen\nen hebbe noch niet gemancqueert\ntwee mael per week te schrijven,\nniuws isser jegenwoordich gansch\nweijnich dan dat het parlement\ndagelijcks vergadert over de geltmidde-\nlen, en sijn voorleden Vrijdach d'or-\ndinaire Revenues voor  s'Koninks\nlenen geconsenteert, uijtgenomen\n\nDe costuijmen die maer voor vier\njaer sijn toe gestean, allen gelooft\ndat de konink Jacob in ijerlandt op\nde defensive sal gaen en dat een\ngedeelte vant Rijck daer toe\nper avantasieus is soo dat sommige\nbeduchten die Conqiustre soo samen\nniet vallen sal als men weergeeft\nechter moeten uijt contrarie\nhoopen, en gaet men met een\nschoone armee derwaerts\nsal U.H.Ed dit mael niet lange\nop houden, maer met officiers\nvan mijn gehoorsame dienst aen\nvrou van Amerongen\nblijven etcetc\n\nMonsieur\nMonsieur le Baron de\nReede Signeur  d'Ameron-\ngen,Ginckel ,Elst etc\nA’La Haije\nWestminster den 25 april\n1690\nIck gaf mijn d'eere mijnen laeste uyt\nhet Huys van den grave van Solms daer\nwij in goedt geselschap vrolijck waeren aan\nU.HEd: met der haest  en soo vertrouwe\nwat confuselijken te schrijven waer\nover exuse bidde t'sedert sijn geene brie-\nven uit holland  over gekomen, die wij\nechter alle moment verwachtende zijn\nen verlangen wat vant verraet van Sluys\nbevonden werdt, het is niet bedenckel(lick) dat\nter een man als den Lt. Coll {Luitenent Kolonel) Palms aen-\nvast soude sijn dat hij er soo ick hoore\nt'sedert enigen tijdt seer tot den dronck\nvervallen waerdoor de menschen som-\nwijlen tot quade verrukingen komen\n\nKunnen, morgen sende ick mijne Equipa-\nsie naer Schotlandt vooruijt , en hoope\ndat inde aenstaende weeck sal volgen kun-\nnen, want bent hier seer moede, en\nnu doch de reijse doen moet, soo verlange\nmaer om wech te sijn, en hoope dat het\nniet lange duren sal. Men is en blijft\nhier even seer om gelt verlegen, en geloo-\nve ick dat het expresse werdt gedaen\nom de koninck t'incommoderen, de con-\nventien sijn wel gedraegen maer men be-\nneempt indirectelijcken het gehediet,\nde partije schappen int parlement wer-\nden dagelijcks grooter, en het gaedt verder\ndan men de penne vertrouwen derft,\nsoo dat het hier noch alles niet voor\nde windt gaet, maer het is een Criemen\ndaer van te spreecken. Den Admirael\nHerbert alias Milord Torrington heeft\nsijne commissie willen neder leggen,\ndoch t'is gisteren weder geaccommodeert.\nMen seijdt dat den Grave van Nottin-\ngam secretaris van staet, oock te quite-\nren staet, soo dat hier nu en dan al veele\nverandering voor valt, dat p. en d.\nniet wel sijn is seker, doch t'werdt noch\ngesimuleert, U. H. Ed. sal apparent nu al\nuijt den Haeg vertrocken sijn, en desen\nhem t'Amerongen of elders op de wech\naen treffen. Godt geve U. H. Ed. veel geluck\nop sijne reijse, en dat deselve se in gesont-\nheijt voltrecken mach, ick hoope dat\nhij op sijn wederkomste alles in meerder\n\nrust sal vinden, dan hij het laet maer\nofter inmiddels geen verandering voorko-\nmen sal, leert de tijdt, de Burgemeesters\nRepelaer, en vanden Broeck beijde van\nDort, sijn hier over haere questie met\nAlewijn en sullen soo ick hoore met\nsatisfactie vertrecken, se hebben soo al\nstoudt gesproocken en krijgen het daer\nom naer haer sin, den den Heere Smetton\ndie over sijn swaeger Flor seer is gealar-\nmeert geweest, heeft mijn noch gebeden\nt'onderstaen of U.H.Ed: iets van sijne dis-\ngratie was voorgekomen dat mijn leet-\nsoude doen om dat niet beter weet\nof Flor is een eerlijk man , en voor goe-\nde partije genegen , U.H.Ed sal apparent\nint Sticht van Munster den vice domi-\nnus\nveelen, den erfkamer Heer Gaelen en an-\ndere goede vrienden sien die ick bidde dat\nvan mijnen dienst mogen versekert werden\nen waermede blijve, etcetera\n\nSt Jeemps 28 apri\n1690\nSA: V;\nMet de post van gisteren hebbe ick\nd'eene gehad twee brieven van U.H.Ed.:\nt'ontvangen beneffens een darde van\nsijn eijgen handt geschreven en of\nwel mijn het bewuste swaer valt\nter negutien mijn echter nae U.HEd:\nadvies  ick segge niet met allen\ndraege het met gedult en sal de\ncampagne doen ten minsten voor\ntijdt dat de Konink zal swesent\nsijn. Het doet mijn leet dat te\nbereets in Hollandt soo veel, van\nte zeggen valt bidde dat U.H.Ed:\n\nhebben, en sijn vriendt en die-\nnaer sijn. s'Gravemoer is goedt\nbij der harten maer het sijn\nwat schope dissolute, Den Heer\nvan Oyen die soo een dichten\nRoes heeft presenteert sijn\ndienst. Het verraet van\nZluijs surprenneert ons\nseer, En ick bidde dat U.H.Ed.\nmijn gelieft te schrijven\nwat is, vande disgratie van\nFox aent Hof van Branden-\nburch dat diest schrijft U.H.Ed.\nhem geseijdt te hebben en\nden armen Smettou die\nMorgen, hebbe ick vande Koninck ordres bekomen\nom mijne Equipages voor af naer Schotlandt\nte senden, alwaer ick te Kirckenbricht tussen\nde drie en vier hondert Mijlen van hier met\nhet meeste gedeelte van onse Ruijterije over\nschepen sal, en heeft mijne equipasie\nbij de maent werck eer dat se daer kompt.\nIck moet op ordre voor mijn parsoon hier\nblijven, en dan te post volgen. Al dit werk\nkost mijn een schoone stuijver doch ick hoop\nhet sal sich ten besten schicken, en het oude\n\nGeen eerlijke man is seeer\nalameert, ick blijve en\nbidde de Vrouw van Amer-\nongen niet qalijck neempt\nhaer dit mael niet\nantwoorden men sendt mijn\neen half dozijn pijpers aen\nmijne ooren die het schrij-.......,\nven beletten\n\nHet wil ignoreren en mijn laet-\nten begaen, Ick hoope dat met\nfatsoen en reputatie daer uijt\nkomen sal maer dan niet meer-\nen ick sal noijt meer mijn met\nijdele hoop flatteren U.H.Ed: bidde\nick dat van gelijcken doet en hij\nsal sien dat die mesure de beste is\nwat haer antwoort U.H.Ed: vande\nKonink krijgen sal op sijn laeste schrij-\nven, wil mijn verlangen, ick bidde\ndat U.H.Ed: niet qualijck gelieft\nte nemen, dat van redenen\ndaer niet naer vraege, vande\n\nDivies in mijne standaeren was post nubu-\nla phoebus, het ongeluck aen onse Vloot\nin Middellantse see gebeurt doet mijn en\nalle eerlijcke lieden hier van harten leet doch\nt'is al godes wil dese morgen heeft de ko-\nninck revue over onze Hollantse guardes\nte voet gedaen dat versaeck geeft ick\nmett den Grave van Solms dezen middach\neete en wij lustich sijn gedachte Heer trac-\nteert mijn met als een vrient maer als een\nBroer waerom bidde dat bij leven of ster-\nven U.H.Ed: altoos voor hem wil ijver theijt\nWestminster den 9/19 mei\n1690\nIck hoop dat U.H.Ed: mijnen brief vande 5e sal\nbehandicht werden waerin verscheijde saecken\ngeschreven hebbe die de post niet toe vertrouwen\nde Men is t'sedert twee daegen int parle-\nment seer besich geweest over het confe-\nreren van Regeringe op de koningin bij absen-\ntie van Konink en het arresteren van\nHet van abjuratie van Koninck Jacob\nhet eene en andere is door gegaen int hoger-\nHuys ter presentie van konink doch met\nweijnich stemmen bij de Vr : overgehaelt en\nsal ick de partikulariteijten daervan pas naes-\nten schrijven kunnen bij aldien noch soo lange\nhier blijve. De F: die inde Regeringe van londen\nnu meijnen de sterkste te sijn hebben voor\nden 26 van deze Maent haeren Lord Major\n\nen de verdere Magistraet jegens aenstaende acties\nvast te stellen omse noch voor des Koninks\nvertreck te doen af?proberen doch of dit geen\nobstackel vinden, wil leert den tijdt, se geven\nvoor dat wanseer se vande superioriteijt inde\nRegeringe voor soo lange versekert sijn, dan\nsullen kunnen crediet schaffen, Daer laeten\nniet swaerhoofdige lieden te sijn, die beduchten\ndat d'abesentie vande koninck, en het vertreck\nvan alle de duijtse troepen uijt het\nrijck wel eene swaere verandering voor-\nbrengen mochte, inmiddels soo hebbe ick\ngisteren ordre bekomen om mijne reijse met\nden eersten voort te setten, soo dat int\nbegin vande aenstaende weeck vertrecken sal\nbij aldien met de penningen klaer raecken\nkan, U.H.Ed: missive vande eersten Maij hebbe ick\nmet de laeste post ontfangen en sal Majesteit\nnu apparent al verre op sijne reijse sijn\nene Godt geve dat wel af loopt, en U.H.Ed\nmet benoegen weder koome Het is mijn lief\ndat mijn vrou en kinderen U H.Ed. op Ame-\nongen hebben opgewacht, ick hoope dat\nonse, en in sonderheijt voor mijn de campagne-\nkort sal vallen, en dat het op sijne weder\nkomste sal kunnen doen de propositie vande R\ngevalt mijn niet te wel, en beduchte ick dat\ndie Muijs een naedelige stardt heeft, dit is noijt\naen d'noch,andere geschiedt waerom sal ment\nint Reguard van U.H.Ed doen, en hebbe ick noch\ngeene reden van andere opinie te sijn, dan dat\nmen U.H.Ed. expresse heeft eloigneeren willen\nen trachten sal eenigen tijdt buijtene s'lants-\n\nte houden, gelijck de tijdt leeren sal, ick\nsal eerdaechs een proef nemen vande continuele\nhoop, en beloften die gegeven werden, en eijschen\nde survivantie vant Jag: Ampt. doch ick twijfe-\nle oft sal werden geaccordeert, den vice\nadmirael Almonde is uijt de Midlantse\nzee ontboden, om met sijne bij hebbende scheepen\nde vlooten te verstercken, doch men\nhoudt het secreet, ick versoecke dat\naen den Heere  Blanche mijnen dienst gepre-\nsenteert werde en blijve etc.\nWat de Heemraets plaetsen aenbelangt ick\nhoope soo lange niet uijt te blijven of dat se\nselver haest weder sal bedienen kunnen,\nen vindt U.H.Ed. dan goedt dat men se eens\nvoor al bij sijn drostampt belijdt daer hebbe\nniet tegen vande honner isser doch niet veel te\ndoen\nWestminster den 15e/25e maij 1690\n1690\nVermits U.H.Ed. op reijs is soo hebbe\ndeselve met de voorige post  niet  geschreven\ninmiddels soo hebbe ick ordres gekregen\nom te vertrecken, en stae noch desen\ndach mijne reijse aen te vangen over Schot-\nlandt met de Heeren van oijen en  en Golsteijn\nMijn Zoon blijft hier, en staet de Com-\npagnie vande Heer van Auwerkercken inde\naenstaende week te marcheren die te\nChester met het Hof over sal gaen.\nGodt hoope ick sal ons alle geluck geven, en\nin gesontheijt weder komen laeten, en salt\nijersche werk wel gaen soot hier niet hapert,\ndaer noch al veele swaerhoofdich over sijn\nde T: werden meer en meer Meester, met\n\nDe Welcke het Hof sich voegt, dat de W.\nRaesent maeckt, en noch al schaersheijt\nvan gelt veroorsaeckt, t'welck niet bete-\nren sal, als de koninck vertrocken is, de\nRegimten te paerdt van s'Gravemoer en\nZuijlesteijn, sullen in Engelandt, d'eenichste\nvan onse Troepes blijven en ontrent\nLonden komen, inde stadt is t' sedert een\ndach a twee veel alarm geweest inson-\nderheijt opde Beurs, dat hier geen meer\nvan onse duijtse troepes blijven, Het\nRendevous van de vloot is verandert en kompt\nde selve van Spithaijt in duijns. de Hollant-\nse scheepen komen traech bij, men gerucht\ndat de fransen sich haesten, om de eerste\nin zee te sijn en dat se voorde Rivier\nvan Londen komen willen, om de conjunctie\nte beletten, ende stadt in consternatie\nte brengen, doch wij moeten hoopen dat se\nsullen werden gepraevenieert, in Ijerlant\nwacht den Hartoch van Schomberch naer\nArtellerije om Charlemont t'attacqueren\nwelcke plaets midden in sijne quactieren\nleijdt, ick geloove U.H.Ed. bij mijne laeste\ngeschreven te hebben hoe dat de Collonel(?) Leve-\nston 1700 Rebellen in Schotlandt aengetrof-\nfen heeft, en totaliter geslaegen, waervan\n400 op de plaets sijn doot gebleven en 100 ge-\nvangens bekomen, ick hebbe de bewuste\nsurviverne gevraecht, doch se is mijn beleef-\ndelijcken afgeslaegen met goede woorden\nals het ordinaris gaet, soo datter niet anders\n\nTe doen is dan mijn naer dese campagne\nhoe eer hoe liever te retireren, ick\nbidde Godt voor U.H.Ed. gesontheijt en\nblijve met haest etc\nwestminster den 18e\n24 April\n1690\nick bidde pardon soo voorleden dijnsdach\neen weijnich confuijs geschreven hebbe, het\ndroevich ongeluck op middachten gebeurt\nheeft mijn soo gealtereert, dat nauwelijcks\nbij mijn selven komen konde doch het is\nGodes wil geweest diese ons geeft, en we-\nder neempt, wij moeten gedult hebben\nen hem bidden voorde conservatie vande\nandere. De koninck heeft sich gisteren\npositive tegens mijn verklaert, en begeert\ndat de campagne van IJerlandt sal doen\nen onder Schonberch staen, ick hadde\neen ander expedient verwacht of dat\nmen mijn naer Hollandt soude gesonden\n\nhebben doch moet dit mael patientie\nnemen, en hoopen dat haest sal kunnen\nverlost sijn van altoos kleijnicheden in\nplaetse van Recumpense voor mijne ge-\ntrouwe diensten t'ontfangen, ick hebbe\nmijn te lange geflatteert met saecken die\nnoijt gebeuren sullen en waerom op andere\nmesures maet gedacht sijn doch hier toe\nis tijdt, en ick bidde dat men daer van niets\nsegge ofte aen laet gaen maer alles onge-\nmerckt passere, mijn is in langen van U.H.Ed\nniet gesproocken, en naer mijn oor deel hoe\nminder empressement U.H.Ed. voor sijne Com-\nmissie gelieft te toonen en der hier over-\nschrijft hoe men U.H.Ed: eerder sal doen gaen\nen beduchte of te veel brieven somwijlen\nNiet verveelen sullen, De saecken gaen\nhier noch niet voor windt, en men blijft seer\nompt gelt verlegen oock soo sullen de bereets\ngemaeckte schulden, en onkosten die dit\njaer te doen staen, de geconsenteerde pen-\nningen verre surmonteren, en werdt de tijdt\ndoorde partijschappen int parlement gecon-\nsumeert, veele sijn van opinie dat den\nkoninck behoorde wat meer vertrouwde\nMilitie in Engelandt te laeten, doch de\nselve sal t'beste sijne mesures weeten\nte nemen Den Marquies van Mompoullan\nsal voor volontaire in ijerlandt dienen\nen soo  geseijdt werdt een pensioen vanden\nKoninck hebben, ick derve niet bidden of\nU.H.Ed voor sijn vertreck den Graef van\n\nSolms en mijn met onse presensie konde\nte recht helpen van Heeteren heeft d'or-\ngineele verklaeringen van vander Esch\nick ben U.H.Ed: onderdanich bedankende\ndat hem vander Esch aen den Heer van\nAchtienhoven met succes heeft gerecom-\nmandeert, en blijve etc\nick bidde noch maels dat desen mach wer-\nden verbrandt, en van't voorgementioneer-\nde niet gesproocken\nR.Hamburg 21 maij 1690\nWestminster den 29 April\n1690\nUijt U.H.Ed. missive vande 22e deses verneme\nsijn vertreck uijt den Haege Godt geve\nU.H.Ed. eene geluckige en voorspoedige\nreijse, en dat deselve met genoegen en\ncontentement Repatrieren mach\nDen konink geliefde mijn desen morgen\nte vraegen naer U.H.Ed. vertreck en\nof deselve sijn wech over Breemen\nnam, ick seijde gehoort te hebben dat\nU.H.Ed. bij den Biscop van Munster soude\naengaen, hebbe verder uijt sijn Majesteit\nniets van U.H.Ed. laest geschrevene brief-\nvernomen oock (om redenen) daer van\nmet veel spreecken derven en beducht\n\nof te veele brieven somwijlen niet\nverveelen, en of men hier het deense\nwerck van soo  grooten gewicht rekent,\ndat echter U.H.Ed. t'beste weten sal.\nDe saecken staen hier noch al wonder-\nlijck, en daer komen dagelijcks donkere\nWolken op, de partijschappen en Cabaeles\nwerden groot, en de Thoris en Wichs\nsoo noempt men de twee partijen\nwerden seer animeus jegens den anderen\nt' welck aende gemeijne saeck seer scha-\ndelijck is, en des konincks mesures\nseer breeckt, den tijdt wil leeren waer\nhet noch termineren sal, ondertussen\nsoo blijven veele saecken bij mangel\nvan gelt ongedaen, en de tijdt verloopt\nIck hebbe mijne Equipasie vast wech\ngesonden maer weete niet wanneer\nse noch sal volgen. Milord Nottingam\neen vande twee secretarissen van staet\nheeft quiteren willen, doch het is\nweder geaccommodeert. Nu seijdt men\ndat Milord Schrosburrij sijnde den anderen\nde U.H.Ed. inde Haeg heeft gesien maer\neen oog hebbende gisteren heeft gequi-\nteert, doch of het mede noch sal\ngeaccommodeert werden leert den\ntijdt, hij was vande morgen lang bij de\nkoninck. Dit sijn de twee eerste Mi-\nnisters, en kan U.H.Ed. dencken dat het\nveel verandering soude bij brengen\n\nsoo bij dese conjuncturen van\ntijden een van beijde sijne Charge\nverlaet, in sonderheijt daer de Ko-\nninck voor sijn parsoon uijt het\nRijck wil gaen en noch al vrij\nveele gemoederen onstuijmich sijn\nDen Admirael Russel is uijt Spaen-\njen met vijf a ses scheepen gerever-\nteert, het vervaerdigen vande vloot-\nwacht mede al naer het gelt, men\nheeft voorleden zaterdach een niuwen\neet om koninck jacob af te sweeren\nvoor gebracht, maer se heeft niet\ndoor gewilt, ick bidde dat dit alles\nnaer discraetie mach gemenageert\nwerden de doot vande Hartoch van Lother-\ningen is al een groot verlos, ick blijveets etc.\nR: Hamb: 31 Maij 1690\nWestminster den 2/12e Maij 1690\nU. H. Ed. missive vande 5e Maij uijt Amsterdam\ngeschreven is mijn gisteren behandicht\ndie mijn het voort setten van sijne verde-\nre reijse vernemen doet, ick hoope dat U.H.d\nallenthalven met benoegen sal ontfangen\nen bejegent werden en dat ick haest sijn\ngeluckich arrivement soo aent Munsterse\nals deensche Hof sal mogen verstaen\nIck hebbe aent Munsterse Hof noch wel\naffaires doch weete niet door wat wech\ndaer uijt te raecken, den Drost van\nAhuijsen wil niet accorderen om dat\nhij ons goedt besit, den Heer van Lembeeck\nheeft ons vande winter eene niuwe\nquestie gemoveert, en begonnen met\n\nHet arresteren van onse Alentse goederen\nsonder dat ons eenige Citatie te vooren\nheeft gedaen, of sijne presensie weeten\ndoet, soo sijn wij oock gequelt geweest\nmet sekeren trompetter Mutschaert\ndie oock al doort toe doen van drost\nvan Ahuijs soo ick vertrouwe sententie\njegens ons gretetineert heeft, en alsoo\ngeen tijdt hebbe gehad van ninter iets\nin saecken te doen soo hebbe het den\nsecretaris otters, en doctoor Vrijden\nbuigge van ootmaersen aen bevolen\ndie hoope datter in vigileren sullen\nen sal den eersten U.H.Ed: ongetwijfelt\nhebben geinformeert soo ontrent\nden Bischop daer in niets te doen is\nDen vire dominus veelen en den erf\nkamerheer Gaelen sijn mijne vrienden\nen souden haer best wel doen tot\neen accoort met den drost van Ahuijs\nmaer men moet sich van mijne kant\nniet te veel aengaen laeten, en\nwaere ick het echte met hem eens\nsoo soude de rest wel gaen U.H.Ed: heeft\nuijt mijne voorige gesien hoe het hier\nstaet t'is niet beter maer erger\nen Milord Schrofburrij die maer een\noock heeft, en U.H.Ed: in Haeg en tot\ndijckfelts voor desen heeft gesien, blijft eerst\ntot noch toe quiteren, de charge van\nsecretaris van staet, welck ampt over\nde m/10 sneerlings s'jaers doet,dat hollants\n\ngelt hondert en tien duijsent gulden beloopt.\nMen doet echter groote devoiren om\nhem te doen blijven, en soo het niet luckt\nsal Milord Nottingam d'eerste wesen,\nen komen tot de tweede plaets twee\nparsoonen in consideratie, te weeten\nsekeren Willamson die voor desen\nsecretaris van staet is geweest, en heer\nRobbert Soutwel die U.H.Ed. voor desen\nsoo Famillair aent Barlijnse Hof heeft\ngekent, en mijn altoos seer naer sijne\ngesontheijt en welvaeren vraecht,\nsoo ick hoore is hij een Swaeger van\nMilord Nottingam voor desen genaempt\nFinckes, dit is de partije vande Thoros of\nDe Kerck van Engelandt, daer het Hof\nnu voor schijndt te pancheren en die\nbij gevolge boven leggen, dat de Wichs\ndie presbiteriaenen, en non Conformisten\nsijn, seer chagrijneert, en tot extremi-\nteijten brengt, meijnende dat sij de Koning\nhebben ingehaelt, en op den troon gebracht\nen daerom het grootste Crediet behoor-\nden te hebben. Ja de partijschappen gaen\nsoo verre, dat d'animositeijten tussen\nhaer veel grooter, dan tussen ons en\nde Rooms gesinden is, ende Nonconfor-\nmisten of presbiteriaenen sijn de consi-\nderabeltste in Londen, en hebben de Beurs\ndie soo danich geslooten werdt datter\nbij negotiatie het welcke hier de manier\n\nis te doen op de penningen die geconsenteert\nsijn, niet een stuijver is te krijgen, en\nbij aldien men sal wachten moeten\ntot dat dit gelt in kompt sal men\nnoch Maenden wachten moeten, behalven\ndat geseijdt werdt, buijten een niuwer (???)\nconsent daer aen drie vijfde parten sal\nte kort sijn. Godt geve dit alles wel\naf loopt en ick hoope dat om dat de\ndispuijten soo hooch loopen het accom-\nmodement des te eerder sal gevonden\nwerden, mijn oude divies dat placht te\nsijn post nubula phoebus, den Grave\nvan Solms, en ick bennen U.H.Ed. seer\nbedanckende voorde goede officien die deselve\nbij den Here Witsen voor ons heeft\ngedaen, wij willen hoopen het een goedt\neffect sal hebben, ick sal mijn d'eere\ngeven te continueren het schrijven\nvan mijne brieven over Hollandt tot\ndat een nader Addres hebbe. Den Heere\nAlefelt die vande Croon deenmarcken\nhier is, doet mijn veel beleeftheijt\nen seijdt U.H.Ed. in Saxen gekent te\nhebben, ick versoecke dat den Heere Blan-\nche mijnen dienst hier gelieft te vin-\nden, en blijve etc. den Baron de\nBongaert versekert U.H.Ed. van sijn\ndienst, soo als doet den Heer van oijen\nen alle goede vrienden\n\nbedanckende voorde goede officien die deselve\nbij den Heere Witsen voor ons heeft\nRec. Hamburg: 31 maij 1690\nWestminster den 5e/15e Maij 1690\nIck hebbe d'eere gehad t'ontfangen met de\nlaeste post U.H.Ed. missive van Amerongen\ngeschreven, die mijn vernemen doet dat\ndeselve vrijdachs daer aen sijne reijse vermeijnde\nvoort te setten, apparent soo sal U.H.Ed. nu\ndan al verre sijn geavanceert, men vindt sich\nhier een weijnich verlegen, dat den cheurfurst\nvan Brandenburch selfs in parsoon weder\nte velde wil gaen, en te gelijk praetendeert\nhet commande over de Keijserlijcke Armee\ndat Beijeren mede praetendeert, hoe dit alles\nsal werden ingeschickt leert den tijdt, oock\nof de Munsterse troepes bij de Brandenburchse\nof Keijserlijcke Armee ageren sullen\n\nMen heeft hier gisteren oock tijding gekregen\ndat Sweeden de commercie met Vranckrijck\nheeft open geset, en in sijn landt publiceren\nlaeten t'welck U.H.Ed. in sijne Negotiatie niet\nnaerdeelich wesen sal. En gelijck ick desen\nbrief met een particulier vriendt tot in\nHollandt sende, soo sal U.H.Ed. een weijnich\nparticulariseren hoe de saecken hier staen.\nIck hebbe U.H.Ed. voor desen geschreven van twee\npartijen d'eene werden de Wichs en d'andere de\nThoros genaempt, de Wichs sijn de presbite-\nriaenen, en eenige vande Engelse Kerck, die sich\nechter met de Thoros niet confirmeren\nwillen, en die geene die ons hebben inge-\nroepen, ende Koninck tot de croon gebracht\n(soo se voor geven) oock soo is onder haer\nHet meeste contante gelt, en sijn de mach-\ntichste van Londen vandie partije, de Thoros\nsijn het meerendeels vande Bischoppen, veele\nvoornaeme Lordts, en als geseijdt hebbe die\nvande Engelse kerck, dese hebben groote\ngoederen maer sijn soo machtich in contanten\nniet, en gelijck de Wicks int voorige parle-\nment de sterckste waeren en soo geseijdt werdt\nniet alleen wat insolent wierden, maer tot\nverandering van Regering en eene Republiek\ninclineerden, heeft de koninck goedt gedacht\ndeselve te dissolveren en een niuw parle-\nment te beroepen, hier heeft nu een ijder\ngecuijpt om weder geaeligeert te werden\nen of wel de Thoros de sterckste geworden\nsijn, soo is echter de partije vande Wichs\n\nDoch soo groot, dat se d'andere bij nae Equaleert\nen werden se soo amineus tegens den anderen\ndat het een schrick is, d'eerste sluijten. soo-\nveel doenlijck haer Beurs, beletten datter\ntot noch toe geen gelt genegotieert kan\nwerden, op voorgeven dat se onder het gou-\nverno vande Thoros niet kunnen secuur sijn, en\ndat koninck gaat daer onder veele toege-\ndaene en vrienden heeft, en waerom de Wichs\nvoornoemt een Eet hebben voorgeslaegen van\ngetrouwicheijt voor koninck Willem, en\nabjuratie van konink jacob die sulke expres-\nsien heeft, dat d'andere tot noch toe daer\nniet aenwillen eergisteren is het Hooger\nHuijs tot negen uren inde avont hier over-\nvergadert geweest, en sijn daer groote Cavella-\nhier\nvan gelt, en sijn wij al over de drie Maen-\nden ten achteren, en d'engelse noch meer\noock soo kunnen de consenten die gedraegen\nsijn op verre nae, d'onkosten niet toe-\nreijcken inmiddels soo verloopt het saisoen\nen men sal in IJerlandt naer september niet int-\nvelt staen kunnen om de Regenmaenden\nhet geene hier schrijve bidde uw dat geme-\nnageert mach blijven en ick sal voortaen de\nWichs en Thoros bij W en T noemen soo-\nwert mijn gesegt dat de koninck vande weeck\nvier hondert duijsent lb steerlings toucheren\nsal, daermede sal ongetwijfelt onse Marsch\nvoort gaen, ick hebbe U.H.Ed: dit int Breede\ngeschreven op dat deselve vant geene hier\npasseert geinformeert, mach sijn, doch soude\n\ngevallen, echter soo is men onverrichter saecke\ngescheijden den tijdt sal leeren oft van daech\nwerdt hervat, int ooch vande Wichs loopen seer\nMilord danbij of Camarden genaempt, een\nden secretaris van staet Milord Nottingam\nswaeger van Soutwel welcke beide seer het\noor vande koninck hebben, en dat groote jalousie\ngeeft aen Milord Mordan Schrosburrij, den Har-\ntoch van Bolton die nu niet minder dan geck\nis en veele andere U.H.Ed doch onbekent, en\ndie de partije vande Wicks houden men doet\nechter noch groote devoiren om Schrosburrij\naende handt te houden, en niet te laeten\nquiteren, want der op hem meer volgen souden\nondertussen soo staet alles stil en kan de Vloot\nniet in zee noch d'Armee int velt bij mangel\nHet met de post niet derven doen om\ndat de brieven dickwils geopent werden\nde laeste pacquet Boot is door een frans\nfregat geattacqueert geweest, en heeft\nscherp gevochten doch is niet genomen\nDen Heer vande lier die soo van mijn gaedt\npraesenteert sijn dienst aen U.H.Ed: en ick\nblijve etc.\n\nRec. (opte: 14 Junij 1690\nYorck den 21/31e Maij 1690\nIck hoop dat desen U.H.Ed: niet verre van\nCoppenhaven aen treffen sal ter waere dat\nU.H.Ed. sich tussen wegen heeft op gehouden\nick ben hier oock gisteravont gearriveert\nen beginnen onse troepes te marcheren\ndat de goede geintentioneerde met\nbekommeringe sien die noch al voor dis-\nordres vresen wanneer wij sullen wech\nsijn doch Godt kant voorsien die aende\nKoninck wijsheijt geven wil ick sal\nhier twee a drie daegen staen blijven\nom noch eenge ordres af te wachten\nen dan mijne reijse op Carlijlle en\nKirkubeits voort setten op welcke\n\nlaetste plaets de Zee passeren sal de\nHeeren van oijen en golsleijn die hier bij mijn\nsijn presenteren haer ootmoedigen dienst\naen U.H.Ed. en ick blijve etc\n\nMonsieur\nMonieur le Baron de Rade\nSigneur d'Amerongen Ginc-\nkel, Elst, Lievendael, etc\nChevaller de t'ordre de\ndenemark etc\nrec  26e dito\nCarlijell den 2e/12e Junij 1690\nUijt Yorck hebbe ick mijn d'eere gegeven laest\naen U.H.Ed. te schrijven, ben voorleden dins-\ndach van daer verreijst, en gisteravont sijnde\nsondach hier aengekomen, de wech is\nwat difficiel voor mijn koets en waegens\ngeweest, om dat wij Hoge bergen en\nveele steenrotsen gepasseert, hebben\ndoch ick hebbe noch alles (godt dank wel\nhier gebracht, en ben nu in laeste Stadt\nvan Engelandt en niet meer dan een uur\nvan Schotene grensen wij hebben noch vier\nen twinch uren gaens te marcheren\nom tot Kirkuberits sijn de haven van\nons Embarquemert te komen, en dan\nkunnen wij met de minste goede wint-\n\nin acht a tien uren oversteecken en\nte Karrickfergus alwaer het generaele\nRendevous is, aen landt sijn soo dat wij-\nnu in korten d'ijersche lucht sullen ge-\nwaer werden, hier begindt smalhans\nal keucken Meester te werden en manc-\nqueert ons de Wijn, dat den oversten\nEppinger slecht gevalt, die sijn dienst\nbeneffens den Heere van oijen aen U.H.Ed\npresenteert, ick hoope dat deselve nu-\nte Koppenhagen geluckelt sal wesen aen\ngekomen hebbe mijne laeste brieven van\nU.H.Ed. van Amerongen, Uijt londen werdt\nmijn vande 27e geschreven dat de koninck als\nmorgen ofte overmorgen wesende den 4e of 5e\ndeses van daer vertrecken soude en\nDen vierden dach tot Chester sijn al\nwaer sijne Maijesteiten sal embarqueren\ndach se moeten goede Wint hebben en\nhet traject is daer soo groot als tussen\nHollandt, en Engelandt, mijn soon is bij de\nkoninck, en hoope ick dat buijten onge-\nluck over komen sal voorts soo weete\nvan hier niet veel niuws met te deijlen\nt'sedert dat den Coll. leveston de Schotse\nRebellen laest wat geslaegen heeft\nkomen se niet meer uijt de Bergen\nen sijn seer verstroijt, hier doort Landt\nwaer ick gepasseert hebbe, vinde noch\nveele Misnoegde (Godt geve dat se niets\nentrepreneren gedurende ons uiit sijn,\nwant wierdt dit Godt vernagelt soo sonder\n\nWij eene moeijelijcke terug komste heb-\nben, ick hoop dat Godert U.H.Ed tot sijn\nbenoegen oppast, en dat U.H.Ed: satisfactie\nvan hem heeft, den Heere Blanche hoope\nIck dat onser somwijlen gedencken sal\nwanneer hij aen de knijsnack, en Rostock-\nker Bier sit, waer mede verblijve etc\nheer van Amerongen Dublijn den 10e/20e Julij 1690\nrec: 14e augusti\nIck hoope dat mijne twee voorige brieven\nU.H.Ed. behandicht sijn en dat deselve daer\nuijt heeft gelieven te sien onse geluckige\nen seer avantasieuse victorie op den Vijant\nbevochten, die soo verre naert westen\nvan dit rijck wech, en verstroijt sijn,\ndat men haer niet weder vinden kan,\noock soo is het generaele seggen, dat\nkoninck jacob sich bij Waterfort aen\neen see haeven met weijnich gevolch\nalleen van vijf a ses Menschen heeft\ngeembarqueert, en naer vrankrijck\nis overgevaeren, oock soo seggen som-\nmige dat Lausuijn met hem is, dat ick echter\nqualijck geloven kan en al te slecht\n\nvoor een fransen generael soude wesen\nMen heeft bij t'plunderen van s'Vijants\nBagasie veele pampieren bekomen en ver-\nscheijde secreeten, en correspondentien\nontdeckt, t welck aende Engelse gevangen-\nhuijsen weder neering geven wil. Voorleden\nsondach dede den koninck sijne intrede in\nDublijn men kan licht dencken wat vreuch-\nde dit aende Arme protestanten gaf,\ndie soo lange in kercken, en gevangen-\nhuijsen hadden vast geseten, en dagelijcks\nmet het verwoesten van haere goederen, en\nde doot wierden gedreijgt, Wij sijn gisteren\nmet onse Armee de stadt van dublijn\ngepasseert, en sullen morgen verder naer\nWaterfort op Marcheren om den Vijant\nop te soecken en ons soo verre als\ndoenlijck van alles meester te maecken\nDe generael Lt.  Douglas is met tien a\ntwaef duijsent man naer Attholo Attholo-\nne en de provinche van Connau gedetacheert\nom dien hoeck landts te vermeesteren\nen men laet hier een sterck Guarnisoen\nsoo dat ick hoop wij in korten tijdt hier\nt'werck sullen meester werden, en haest\nnaer ons vaderlandt kunnen overschee-\npen daer d'ongeluckige rencontre van\nden prins van Waldijck en tweijnich\nsucces vande vloot al wat onsteltenisse\ngeven sal, doch ick hoope dat het\nhaest sal gerepareert werden Mijn soon\nheeft de compagnie van Lintelo in\n\nmijn Regimentt gekregen, en Lintelo sal\nweder eene compagnie in Hollandt hebben\nvorders soo sijn alle de vrienden hier noch\ngesondt en doet den Grave van Solms\nmet s'Gravemoer mijn dese middach d'\neere van met mijn te eeten en sullen\nwij U.H.Ed. gesondtheijt drincken ick\nhebbe noch maer eenen brief van U.H.Ed\nvan Coppenhagen gehad, soo U.H.Ed: mijn\niets van dat Hof aen beveelt salt geen\naende Heeren Soutwel en Hop communi-\nceren, ick hebbe vergeten U.H.Ed: voordesen\nte schrijven dat s' vijants laeger al seer\nsterck was en drie a vier en dartich duijsent\nman was Monsieur de la forest diende met\nmijn den dach vande Bataille en dede seer wel\nick blijve etc.\nrec: 18 augusti\nint leger te Clonmel den 23e julij 1690\nIck hoop dat mijnen brief voor acht\ndaegen geschreven U.H.Ed: behandicht is wij\nhebben t'sedert continueel gemarcheert\nen sijn met de Ruijterije tot aen het stede-\nken hier boven genoempt geavanceert, de\nKoninck marcheert met het Gros van\nsijn leger naer Waterfort, een considerable\nSee haven dewelcke men heden door een\nTrompetter heeft doen opeijschen, en ge-\ndreijgt geen quartier te sullen geven soose\nsich opiniateren, de Vijant staet met\nalle haere macht bij lemmerick, haere\nRuijterije op dese sijde vande Channon, en haer\nvoetvolck op d'ander sijde, hebbende de fransen\n\nDe stadt van Limmerick beset, sonder daer\nijeren te willen in hebben, den Vijant is\nweder in een groot getal om dat se alle\nde papisten soo Edelieden, Boeren als vrou-\nwen, en kinderen met een groote quanti-\nteijt van vee bij haer hebben, doch haer\nmancqueert haere Equipage, geweer, en\nsoudt, en of wel se soo veel vee hebben\nmet genoomen alse gekost hebben, soo vinden\nwij echter noch overvloedich, en ist Remarka-\nbel dat se aen een Man in dese stadt 16000 schae-\npen, en wel drie a vier duijsent runderen\nhebben wech genomen, sij hoopen op een\nNiuw secours uijt Vranckrijck, doch wij\ndoen ons best ompt haer te beletten\nen sullen daerom trachten de See haevens\nte vermeesteren, in Engelandt sijn se door-\nde Nederlaech van onse vloot al wat\ngealarmeert, en beducht voor eene\nlanding, en meijnen sommige dat den\nKoninck in korten wel weder naer Engelant\nmocht overgaen, die hier den Winter blijven\nmoet sal een Rude en ongemackelijcke Cam-\npagne hebben, en bij aldien den Vijant sich\nopiniatreert, en door de veelheijt van haer\ngebreck krijgt t'wijfele ick of wij van dit\nsaisoen, vande provintien van Gallouwaij en\nConeck wel Meester werden, ick hebbe\nU.H.Ed. laeste missive den Grave van Pordt-\nland voor gelesen, en versocht den ko-\nninck daer over te willen onderhouden\n\ngelijck den Heere Soutwel mijn oock heeft\ngeseijdt te willen doen over den brief die\nU.H.Ed. aen hem geschreven heeft, doch\nick hebbe t'sedert niet weder aent\nHof geweest, soo dat geen antwoort\nseggen kan ick blijve etc\nrec: 14e aug\nint leger te Timolin den 23e/13 julij 1690\nIck hebbe gisteren d'eere gehad twee brieven\nvan U.H.Ed. uijt Coppenhagen vande 24e junij,\nen 1e julij t'ontfangen waer uijt verneme\nde beleefde bejegeningen die U.H.Ed. soo\nbij sijne eerste Audientien als naederhandt\naent Hof aldaer heeft ontmoedt, en\ndat U.H.Ed. soo veel hoope schept om\nte reusseren in sijne negotiatie daer\nGodt sijn segen toe verleenen wil\nen sal ick mijn bij occasie van U.H.Ed.\nbrieven aen ons Hof bedienen, en die\ncummuniceren daert behoort, wij sijn\nnu met de Ruijterije in marsch vooruijt\nom de schaersheijt van fourage en\nvivres, en sullen in vier a Vijf daegen\n\nNiet weder te samen komen onse marsch\ngaet recht op Waterfort, om ons van\ndaer vande meeste see havens daer de fran-\nsen souden landen kunnen te versekeren.\nHet is noch seker dat koninck jacob\nmet een kleijn gevolch naer Vranck-\nrijck is overgescheept, en dat een gedeel-\nte van sijn Armee sich tot Limmerick\nte samen treckt, dat een vande sterckste\nplaetsen van ijerlandt is, sij ruineren\nhet landt seer, drijven menschen en vee\nwech, en trachten ons de subsistentie\nte benemen, doch wij vinden noch geen\nvee genoch, den drank incommodeert\nhet meest, want het ijersche bier is\nseer ongesondt, en begindt de landt\nSieckte, dat is de loop al inde Armee\nte regneren, ick hebbe noch dorts Engels\nBier en goede Moesel wijn soo lange\nhet duurt, want wij nu van onse\nscheepen afraecken, die tot noch toe\neen groot secours sijn geweest, den here\nDe la foraijs die met mijn aende lincker\nVleugel dient, en seer wel inde laeste\nrencontre heeft gedaen is t'sedert drie\na vier daegen aende landt sieckte onpas-\nselijck, doch soo ick hoore nu weder wat\nbeter, alle de deensche Ruijterije die\nseer goede troepen sijn, sijn aen onse\nvleugel, en kan ick mijn seer wel met\nhaer verdraegen, Hoet met de Bataille\nter See afgeloopen is, sal U.H.Ed. uijt leerdren(?)\n\nHollandt bericht werden, den Engelsen\nAdmirael Herberts heeft het slecht ge-\nmaeckt, en is soo wij hooren op ont-\nboden. Wij sijn gisteren verblijdt te\nhooren dat het verlos in Brabant\nnoch soo groot niet is geweest, bij\nmijnen laesten voor drie daegen geschre-\nven, hebbe U.H.Ed. bericht dat mijn\nsoon nu in mijn Regiment is. Het is\nmijn lief dat Godert sich soo fris houdt\nen dat den Heere Blanche sich soo wel\nmet de Rijnsche Wijn behelpt, dede\nhij eene ijersche campagne, see soude hem\nniet opbreecken, ick sal met parmissie\nversoecken mijnen dienst aen sijn Ed. en\nblijve etc\nRec: 18 sept\nInt Leger bij Lemmerick den 18/8 Augusti 1690\nVoor twee daegen soo hebbe ick weder\nontfangen U.H.Ed. missive vande 25e/15e julij\ngelijck oock drie daegen te vooren een\nvande 18/8 dito. En dewijl sijne Majesteit weder\nvan Doublijn te rugge gekomen is vermits\nin Engelandt alles wel staet, en dat de\nvreese voor het landen vande fransen\naldaer t'eenenmael verdwijnt, heb-\nbende de selve alleen met 3 a 4 hondert\nman in Turbaij aen landt\ngeweest, en aldaer een miserabel dorpjen\nverbrandt, soo hebbe ick occasie gehad\n\nMet sijne Majeste int breede over beijde\nU.H.Ed. missives te spreecken, en aen de\nselve den inhoudt van dien voorgehou-\nden, ick soreech gaft. tot antwoort dat\nU.H.Ed. brief vanden 25e  julij sijne Majesteit\nnoch niet ter handt gekomen was\nom dat alle de brieven vermits de\nselve Engelandt verwacht wierde\nopgehouden sijn doch geliefde deselve\nmijn te seggen, te gelijck dat al voor\neenigen tijdt naer Hollandt geschre-\nven hadde, men U.H.Ed. ampelderlast\ngeven soude, en dat sijne majesteit\nniet twijfelde of U.H.Ed. hadde die bekomen, ick\nsoude niet te min bespeuren datter oock wat\npaenick vallen wijl, over het uijt vinden\nvande penningen, die der sullen werden gerequireert\nvant Hollandt is in geen staet om veele subsidien\nte kunnen geven, en den koninck heeft oock\ngenoch met het sijne te doen soo dat op een\ndarde expedient sal moeten gedacht werden\nten waere dat de somme mackelijck vermindert\nwierdt, voorde rest vonde ick sijne Majesteijt gedispo-\nneert en genegen tot d'aliantie, dit is het geene\n\nin korten tot antwoort te seggen weet, en ick-\nhoop dat U.H.Ed: hier ontrent reusseren sal\nMen schrijft mijn uijt Hollant dat de flauwhartig-\nheijt door onse verkregene victorie seer is verdwee-\nnen, en ick hoope dat men haest alles weder sal\nte boven sijn wij staen noch vijf Mijlen van\nLimmerick, en marcheren morgen dicht onder\nde stadt, als wanneer wij sullen trachten\nof de Stadt t'attacqueren of de shannon\nte passeren soo veel wij van alle overloopers\nhooren sijn de Fransen achter uijt met\nHaer Artellerije en Bagage naer\nGallouwaij geretireert, om haer daer\nt'embarqueren, en isser onder de IJeren\ngroote oneenicheijt, het welcke sich\nin korten openbaeren sal, Kunnen\nwij maer over de Rivier komen die\nbreeder dan den Rijn bij Amerongen is\nsoo sal het werck haest gedaen sijn-\nde vijant heeft gisteren en vandaeg heden\nint gesicht van ons Camp sterck\ngebrandt, dat men haer niet heeft\nbeletten kunnen, om dat het landt\nsoo difficiel, en gebroocken is, doch\n\nAls wij haer morgen naederen soo\nwilt gedaen sijn en ick hoop dat wij\nnoch vande Winter sullen kunnen\nnaer Huijs keeren daer godt geve\nwij U.H.Ed. in gesontheijt vinden mogen\nAlle de vrienden hier sijn noch gesont-\npreasenteren haeren dienst en accom\nmoderen haer tot noch toe, beter met\nd ijerse tucht, dan men inde beginne\ndaer van hadde gerucht, doch wij hebben\noock noch de regen maenden niet , en\nin 30 jaer is hier soo groote droogte niet\ngeweest, ick blijve etc\nrec:  8 september\nInt leger voor Limmerick\nden 12/22 Augustus 1690\nMijnen laesten aen U.H. Ed: is geweest vande\n14/4, deses, waer uijt deselve heeft gelieven\nte sien wat antwoort de Koninck mijn\naengaende de Deense saecken, en op UHE\nbrieven waervan sijne Maj(esteit) een gedeelte\nhebbe voorgelesen gegeven heeft, en on-\nder andere dat U.H.Ed. uijt den Haeg Amp?\nlast stont t'ontfangen die ick hoope\ndat U.H.Ed. ter handt gekomen is Wij sijn\nmet ons leger den 9e hier voor stadt\ngekomen daer den eersten dach wat schar-\nmutselen voor vielen alsoo het garnisoen\nbuijten was, op de Rivier de Shannon\nvonden wij een uur boven de stadt opt\nGuee vijf a ses duijsent man gecampeert\n\nDie men resolveerde den anderen dach met\neen gedeelte van ons voet volck, eenich\nCanon en alle onse Ruijterije t'attacque-\nren, ick vondt mijn weder bij de lincker\nvleugel vande Ruijterije en marcheerde\nmet 't deselve ent Canon, een uur voor\nden dach naerde Rivier alsoo de verste\ndaer van was men hadde des smorgens\nde Resolutie verandert van niet t'attac-\nqueren en wierdt mijn een  adjudant naer\ngesonden doch ik was al op de rivier en\nvondt den vijant geretireert die alleen\neen Trop van 40: a  50 paerden had laeten\nstaen de welcke ick aenstonts dede pocheren\npasseerde de Rivier, en liedt de Konink\nseggen, dat ick gepasseert was, die\nDaerop aenstonts aent water quam, en\nmijn over vondt, waerop men aenstonts\nde passasie met voet volck beset heeft\nen soo vande selve is Meester geworden\nsonder slach of stoot nu hebben wij\ngeen Rivieren van consideratie meer\nte passeren, doch dese stadt soo se haer\nwel defenderen sal noch al menschen kosten\nde fransen sijn met sack en pack naer\nGallowouaij achter uijt geretireert, daer alsvoren\noock het gros van s Vijandts leger naer\ntoe is, doch een partij van haer is eer-\ngisteren tien a twaelf mijlen boven de\nstadt over de Rivier gegaen, en hebben\neen convoij van ons van dublijn komende\ngeattacqueert, en soo vreese eenich\n\nvan ons Canon venaegelt doch wij heb-\nben daer van noch geene sekere tijding\nen ick hoope het sal niet veel te bedie-\nden sijn den Aermen Willem Meester is\ndaer bij en ick vreese voor hem, Dit is\nal t'geene voor dit mael te schrijven\nhebbe, ick blijve etc\nKillabourn den 30/20  Augustus1690\nT'esedert mijnen laesten ben ick met een\ngroote gedeelte van onse Ruijterije en\nDragonders van ons leger gedetacheert\nen de Rivier de Shannon hooger op\ngemarcheert om het passeren vande\nvijandt over deselve rivier te ver-\nhinderen, op dat se ons de schaede niet\nmeer mogen doen, die voor eenige dae-\ngen aen een convoij vande Dublijn komende\nis toe gebracht, waerbij se eenige kruijt\nhebben doen springen, twee halve cartou-\nwen doen barsten, en een affuijt ofte\nettelt verbrant, dat wel van geen\n\nGroot belang is, doch echter het openen\nvande Trancheen twee a drie daegen\nheeft doen stremmen waermede men\nniet te min voorleden sondach heeft\nbegost, en hebben d'onse tot noch toe\ngoedt succes, de Deenen hebben den eersten,\nnacht een werck in gekregen dat van\nconsideratie was, en tot noch toe, als\nniet beter weete werdt gemaintineert\nonse Baterijen hebben gisteren en\nvan daech sterck geschooten, en soo wij\nnoch acht a tien daegen droog weder\nhouden mogen soo hoope dat wij vande stadt\nsullen meester werden of schoon daer\neene kleijne Armee binnen leijdt\nEn van't nemen van dese plaets\ndependeert het gansche werk, want\nsoo wij se krijgen kan den koninck\neen hoopen troepen voor de Winter\ndoen over gaen, en het verdere met\nweijnich volck meester werden, maer\nsoo wij de stadt niet krijgen soude\nmen om de winter quartieren te\nnemen verre moeten te Rugge gaen\nde fransen lichtelt moet geven om-\nniuw secours te senden, en daer aen\ndan noch eene campagne spenderen\nmoeten, twelck voorde gemeijne saeck\nniet goedt soude sijn wat niuws ick\n\nsoo aenstonts uijt s'vijants leger\nontfang gelieft U.H.Ed hier nevens\nte vinden voor mijn vertreck soo hebbe\nmet sijn Majesteit noch int breede over-\nde deense saecken gesproocken en U.H.Ed\nbrieven voorgelesen, ick vinde den\nkoninck genegen doch t'wil opt gelt\naen komen, en seijde sijne Majesteijt mijn\ndat U.H.Ed. nu sekerlijck volkomen\nordres hadde, ick verneme mede dat\nden Hartoch van Wirtenberch bij den\nkoninck niet onaengenaem is om het\nCommande te blijven behouden in kas\nhet werck aen gaet en dat U.H.Ed: reus-\nseert, hij is mijn een seer groot vrient\nen obligant toe waerom de vrijheijt\nint leger te Kerkelis den\n12/2 september 1690\nIck hebbe mijn d'eere niet geven kunnnen\nt'sedert acht daegen aen U.H.Ed: te schrij-\nven vermits ick van ons leger ben gedeta-\ncheert geweest, om een kasteel wech\nte nemen daer een guarnisoen van 80 man\nin lach, en waer in hebbe gereusseert\nondertussen soo heeft men voor lemmerick\ntwee attacques gedaen, en in eerste een\nFort genomen dat gemaintineert bleef\ndoch soo als men van dat fort eene lingne\nvan communicatie tot aen de Transcheen\nmaecken wilde viel de viant te voet, en\nte paerdt uijt, en dede een van onse Batail-,\nlons te voet wat reculeren, den ouden\nBonkourt hadde juijst met driehondert\n\npaerden de wacht aende Trancheen die\nd' uijt vallende soo vigoureus repouneerde\nen weder inde stadt dreef haer vervolgende\ntot inde poort, en blijvende verder voor de\nselve staen, tot dat ons voet volck sich\ngedeckt hadde, dat een ijder seijde noijt\naen geen Ruijterije te hebben sien beter doen,\nhij hadde ontrent de hondert soo dooden als\ngequetste, en wel 130 paerden, hij wierdt sel-\nver min ofte meer gequetst, en den Heer\nvan Niuwenhuijsen die daerop hem quaem\naflossen, wierdt oock gequetst, doch se\ngaen beijde al weder uijt, drie daegen daer\nnaer wierdt de Contre scherp geattacqueert\ndie men aenstonts meester wierdt, en den\nvijant door de Bresse sich retirerende, soo volchde\nd'onse doch wierden op de Bresse en\ninde straeten vande stadt daer sommige\nal in waeren soo bejegent, dat se weder te-\nrug komen mosten, en hebben wij soo dooden\nals gequetste doch meest gequetste 1800 man\ngehad, waeronder verscheijde officieren\ndoch van U.H.Ed. kennisse is niemant doot,\nals Chavernaij en den Armen Cornelle\ningenieur is swaer gequetst, ick hebbe\nhem van daech doen wech brengen en mijn\nChirurgijn met gegeven of hij noch te\nsalveren was, naer dese Attacque soo\nheeft het drie daegen aen den anderen seer\nsterck beginnen te regenen, t'welck den\nkoninck heeft doen resolveren de bele-\ngering op te breecken, en is sijn Majesteit oock\neergisteren naer Waterfort om van\nvan daer voort naer Engelant te gaen\n\nvertrocken, willende dat ick hier blijven soude\nom de Ruijterije te commanderen en heeft\nSchonberch met genomen, ick hebbe alle\nbedenckelijcke middelen aengewent om mijn\nhier van t'excuseren doch het heeft niet\nhelpen mogen, en men heeft mijn .......\naengesonden, en selver de grootste belof\nten ...... recompense se gedaen, die men\naen imandt seggen kan oock soo wil\nmen mijn hier te lande eene considera-\nble confiscatie geven, doch ick hebbe\ngeen genegentheijt tot een anders goedt-\nen oock geen lust, om, een van mijne\nkinderen in dit landt te setten, dat wel\nschoon, vet, en overvloedich van alles\nis, maer de menschen deugen daer niet\nick bidde dat om redenen van dese offres\nAen niemant in sonderheijt niet aen\nden Heere Molesword mach gesproocken\nwerden een tweede saeck die mijn in desen\noock seer embrasseert, is, dat den Grave\nvan Solms quiteren en niet langer die-\nnen wil en inde naeste Maendt te vertrecken\nstaet, hier mede soo ben ick d'ouste en\nde naeste tot het commande van't gan\nsche leger, het welcke te cederen aen\neen jonger soude niet gaen doen, en echter\ndese swaere last op mijn te nemen, daer\nmen de verantwoording van alle Evene-\nmenten aen een Engels parlement doen\nmoet, die met des menschen leven\nspeelen als de kat met de muijs dat\nis oock van een groot belang en naerden-\nken soo dat in groote verlegentheijt\n\nBen. Godt hoope ick sal mijn goede gedach-\nten en raedt geven hoe hier uijt te\ngeraecken, den Grave van Solms en\nick wij hadden dese Belegering niet gaern\ngesien, daer 12000 man inde stadt was, die\nmen niet besluijten konde, en alle daegen\nvant Volck in kreeg, en meijdden dat\nmen eenige andere plaetsen moste wech\nnemen en dese doen creaveren, doch nu\nis het te laet, en salder noch eene cam-\npagne met moeten werden door gebracht??\nondertussen sal mense soo veel benaunen\nmoeten als mogelijck en de Winter quartieren?\nop haer neus nemen ons leger is anders\nnoch in goeden staet, en sonder considera?-\nbel secours uijt vranckrijck sullen sij\nBeswaerlijck bestaen, de fransen sijn\nnoch in Gallouaij en leggen daer 18 schee-\npen van oorloch om haer afte haelen\ndoch ofse nu vertrecken sullen leert den\ntijdt, U.H.Ed: missive vande 15e hebbe ick\nontfangen, en hoope dat nu alle\nmijne brieven, en t'geene daerbij over\nde deense saecken geschreven hebbe\nwel overgekomen sijn, ick twijfele\nniet of den Heere Sautwel heeft oock\nop U.H.Ed. brief al voor langen geant-\nwoort, ick moet in haest afbreecken\nen blijven etc.\nAlle U.H.Ed. brieven komen wel over\nen ick schrijve oock ale weeck maer\nse sijn lang tussen wegen\n\nrec: 17e. november\nKilkennij den 10-20 oct? 1690\nMet de laeste post hebbe mijn d'eere\nniet geven kunnen aen U. H. Ed. te\nschrijven, vermits de kortheijt van\ntijdt, nu sal alleen seggen dat\nKorck en Kincksaijll gereduceert\nsijnde, te Kingsaijll noch den fort\nresteert dat men jegenwoordich\nbesich is t'attacqueren, en of het\nquade weder onse troupes desoleert\nsoo hoope echter dat se het sullen\nmachtich werden, den Heere Tettou\nheeft het andere fort door de\nbanck het oude fort genoempt\n\nStormenderhandt ingenomen,\nen de helft van't guarnisoen gera-\npotheert het selve bestont uijt\n450 man, ick hebbe eergisteren\nde overige Troepes soo noch bij mijn\nhadde naer haere respective Win-\nter quartieren gesonden want wij\nhebben dagelijcks soo quaeden weder\ndat onmogelijck niet langer konden\nblijven staen, en veroorsaeckte desel dese-\nve veele sieckte, ick vreese dat\nons detachement van korck en\nKingsaijll oock veel lijden sal\nIck hebbe nu in langen geene\nBrieven uijt Hollandt gehad\nbijgevolge niet van U.H.Ed: die ick\nhoope , en bidde Godt dat lange jae-\nren bij gesontheijt noch mach\ncontinueren en blijve etc\n\nrec: 24 nov\nArdtinan Brits den 25/15 oct 1690\nDen laesten brief die ick d'eere heb-\nbe gehad van U.H.Ed. t'ontfangen\nis nu wel seven weecken oudt, ick\nhoope echter, dat U.H.Ed. in goede\ngesondtheijt is, en sijne negotiatie\nmet benoegen voltrekt, ick hebbe\nvier daegen geweest te Kilkennij in\nmijn winter quartier, maer den\nvijant met eenige troupes naer\nKingsaijll naederende, daer het niu-\nwe Fort meer spels geeft als\n\nMen hadde gemeijndt, soo hebbe\nhier ontrent weder eenige Trou-\npes te samen getrocken, doch den\nvijant is naer lemrick geretireert,\nen heeft sijn lust geboedt met\neenige dorpen inde asche te leggen\nsoo dat ick hoope wij eijndelijck\nnoch meester vant gemelte\nFort werden sullen dat niet te\nmin al eerlijcke Lieden kost, on\nder anderen is den Hartoch van\nGrafton soon van koninck Carel\nde tweede gebleven en dat\nons het meeste incommodeert is\ndat wij soo veele siecken krijgen\nhet doet niet dan Regenen, en\nwaijen, de Tenten kunnen niet\nresisteren, en U.H.Ed kan dencken\ndit moet groote siecktens veroor-\nsaecken en daer sijn geene dorpen\nals in andere landen waerin\nmen logeren kan, ick dancke\nGodt dat met mijn soon noch\nsoo wel ben, hij praesenteert\nsijn dienst onderdanich en ick\nblijve etc.\n\nHr van Amerongen\nFhethert den 30e sept. 1690\nolden stijl\nNaer dat ick in drie weecken geene brieven\nuijt Hollandt hadde gehad soo sijn mijn,\ngisteren drie pacquetten te gelijck toe\ngekomen, waer onder eenen van U.H.ed.\nvande 9e september apparent niuwen stijl die\nmijn met vreuchde de continuatie van\nU.H.Ed: welwesen, en gesonheijt laet sien\nen dat deselve met soo veele genaede\nen benoegen bij sijne Majesteit van\nDenemarcken continueert te sijn\nGodt laete het continueren en\ngeve dat de selve met gesontheijt\n\nRepatrieren mach, dat ick van harten\nwenste oock te mogen doen of wel\nnu jegenwordich d'eere hebbe sijne\nMajesteits. Armee hier in ijrlandt te com-\nmanderen, en geene van mijne confra-\nters de minste difficultheijt moveren\nom mijn het selve te cederen den\nprince van Wirtenberch de voorste\nvan allen diet op d'obliganste manier\nvande Werelt heeft gedaen, Het laet\nechter niet eene groote bekommering\nte geven, en ben geoccupeert nu\nbij naer nacht en dach, want\nwij hebben over de 80 uuren gaens\nfrontieren te besetten, en buijten het\nresterende van Koning Jacobs Armee\ndie noch al numereus is soo sijn meest\nalle de papisten inde waepenen\ntegens ons, en of wel het misera-\nbel en slecht volck is, soo moet\nmen jegens haer op sijne hoede sijn\nen werden dese Rapparies genaempt\ndaer van onse Major Philintinghof\nlaest met hondert en vijfentwintich\npaerden tussen de drie en vier\nduijsent overvallen, en geslaegen heeft\nen vijf hondert vande selve gecapotheert\n\nnu heeft U. H.Ed: uijt mijnen laetsten\ngesien dat onse vlooten voor Cork\nen Kingsaijll sijn en dat den Har-\ntoch van Wirtenberch ende Lt. Gener-\nrael Malborough die die plaetsen\nattacqueren sullen, terwijle ick\nhier noch met eenich volck\nblijve staen om den vijant t'obser-\nveren, en staen d'Engelse met den\ngenerael Lieutenant douglas bij Atloon\nom soo wel als ick alle incursien\nte beletten soo aenstonts kompt,\neen gerucht ja selfs een brief van\nGoor, dat Corck sich den 28e heeft\novergegeven en dat het Garnisoen\nsijn prisonniers de Guerre tot seven\nRegimenten te voet, en verscheijde\nparsoonen van qualiteijt gemaeckt\ndoch ick hebbe hier van noch\ngeene expresse, en derve het\ndaerom aende koning niet schrijven\nmaer soo dit waer is soo is alle\nhet ongeval van Lemrick gerepareert\nen sullen wij met Godts hulp den\nVijandt desen Winter soo vast over-\nde Shannon sluiten dat se creveren\nsullen en in korten sich over\n\ngeven Het doet mijn leet U.H.Ed\ngeene nader ordres omtrent  sijne\nNegotiatie opt afgaen van sijnen\nbrief aen mijn ontfangen hadde\nick wil hoopen dat se nu gekomen\nis, en U.H.Ed. deselve met benoe-\ngen sal perfecteren, Monsieur. de\nla  Forest is t'sedert eenige daegen\nseer met het graveel gequelt ge-\nweest\nNaer desen geschreven te hebben\nsoo kompt de geluckige overgang\nvan Korck en Kingzaijl als U.H.Ed:\nuijt de nevengaende sal gelieven\nte sien, dat eene groote victorie is\nDen Kappelaen van den Bischop\nvan Killalow kompt met ordre van\nden bischop,  soo aenstonts, aen den\nheere van Ginckel rapportteren\nhet naevolgende\n1-Dat hij Kappelaen voorleden Sondagh, Nae\nmidddagh om vier uijren uijt het leger van\nden Vijant vertrokken is het welk doen\nter tijt bestaende uijt 6 a seven duysent man\nsoo Ruijterie als dragonders en alleen twe\nregiment te voet gecampeert stondt te\nGortinchgorij toebehoorende aen een edelman\ngenaemt ochagnisij twe mijlen van\nkillcolgan twaelf mijlen van  Gallowaij en\n22 Mijlen van Killalow dat hij op sijn\nvertrek hadde gesien int gedachte leger\nTirconnel en Saxfieldt\n2 ende dat de fransen 2 mijlen van Galloway\ngecampeert stonden\n3 Dat in de baeij van Gallowaij waren gearriveert\nomtrent 50 Fransen vaartuijge om gedachte franssen\nte embarqueren\n4.ende dat aldaer alle Momenten een esquad(ron)\nvan 18. Franssen oorlogh schepen verwacht\n\nWierden, om haer nae Vrankerijk over te\nescortteren\nDoch dat Tirconnel aen den Gouverneur\nvan Galloway verboden had haer fransen\ntot het embarquement  in de Stadt te laeten\nen dat hij alle  mogelijke devoiren aanwende\nom haer en door Dreijgementen goede\nwoorden, bij hem te houden\nEn dat het  Gen(era)el seggen in haer armee\nWas dat  bij aldien sij  de franssen konden\nbij haer houden Wilden op athlone,en\nVoorders van daer nae. de provintie van\nulster marcheeren Doch dit Merijteert\nweijnigh gelooft\nVorders soo rapportteert gedachten\nKappelaen dat een groot deel  Rapperijs\nin de provincie van Connaught bij den\nanderen sijn en dat Tirconnel all doet\nwat hij kan om Noch Meerder getal bij\nden anderen te brengen\nDat soo groote Menighte van vee in\nGallowaij is gevlucht, dat het selve\nonmogelijk geen drij weecken daer meer\nkan Supsisteeren\nrec 27e october\nClonwell den 20/30 october 1690\nT'sedert mijnen laesten, soo hebben\nonse Troepes het fort te  Kingsaijll\nCarels fort genaempt, weder ver-\novert, hier mede is de Koninck\nMeester niet alleen vande twee beste\nHavens van dit Rijck maer van\ngansch  Europa en die juijst tegens\nvranckrijck over leggen de IJeren\nhebben nu noch vier plaetsen\nende provintie van Konach, de\nfortressen sijn Lemrick , Galloway\n\nAtloon en Slegoo onse Armee is\nwat seer geharasseert, en wij hebben\nveele Siecken souden anders noch\nweleen van twee laeste plaetsen\nnemen kunnen, ick hoope echter\ndat wij het werck vande Winter\nsoo nae brengen sullen dat\nhet d'aenstaende somer haest sal\nwerden geacheveert, de Deense\ntroepen hebben haer voor Korck\nen Kingsaijll seer gesignaleert\nen in sonderheijt haere generaels\nden Heere Hartoch van Wirtenberch\nende Generael Majoor Tetton\nen hebben wij weijnich volk voor die twee\nplaetsen verlooren, dan den Hartoch van\nGrafton, en seker Majoor vande Deenen\nalle de troepes genaempt,\ngaen nu een op meijnich naer de winter quar-\ntieren om haer wat te refuscheren en\nhebben deselve al veel geleden, ick sal\nvoorde drie compagnien van mijn Regiment\nwel 40 man noodich hebben, en twijfele\nofse in Hollandt sullen te bekomen sijn\n\nmijn soon presenteert sijn gehoorsaem respect\nen ick blijve etc\nWij mancqueren nu seven posten en sijn\nonse laeste brieven van Londen vande 30 September\nolden Stijl, die van U.H.Ed. sijn noch een Maent\nolder\nNeme U.H.Ed. oock het intrest van\ngedachten hee? te recommanderen\nick stae morgen weder een marsch\nhooger op te doen en sal mijn meer\nen meer van ons leger Eloigneren\nsoo dat niet weete of U.H.Ed. soo lange\ndit duurt soo regulier met mijne\nbrieven sal kunnen op wachten\nen het niuws van ons Hof kunnen\nmede deijlen, voor Beusechem sal\ngaern mijn best doen doch beduchte\noft dese mael sal lucken kunnen\nvermits den koninck meeste alle\nde Charges door de laeste Bataille te\nFleuraij vacant gevallen opt voorste\n\nvan de prins van Waldijck vergeven-\nsal Ham maeckt groote apparentie\nom wegens de stadt van Aernhem\nin plaets van Everwijn inde Staeten-\nGenerael te komen. Mijn soon presen-\nteert sijn gehoorsamen dienst, en\nick hoop dat de comportementen\nvan mijn soon van Hervelt U.H.ed\nbenoegen geven den Heer Blanche\ngelieft van mijnen dienst versekert\nte sijn en ick blijve etc.\nWillem Meester ist naeckt ontkomen\nals laest het convoij vande Vijant wiert\novervallen\nHeer van Amerongen\ndomstiek\nOver de saecke U.H.Ed: commissie\nin den Raet van staeten raeckende,\nhebbe ick voorleden Maert sijne\nMajesteit. gesproocken, die de verwisse-\nling approobeerde, maer ik meijn-\nde dat U.H.Ed te Maij noch\neen jaer hadde. sal echter nu\noccasie nemen daer over weder\nte schrijven en sijne Majesteit te\nbidden dat bij sijn arrivement\nin Hollandt het tot U.H.Ed. be-\nnoegen dirigeren wil ick hebbe\nhier een swaere last op den\n\nhals derve alles niet te schrijve\nmaer wenste daer van ontslagen\nte sijn, ben niet alleen met het\ncommandement maer de betae-\nlinge vande Armee, en alles gechar-\ngeert, en het gaet ter al met\nals in Hollandt veeltijdts man-\nqueert veel, ick hoop Godt sal\nmijn redden en en ick blijve etc.\nDrinckt dickwils U.H.Ed: gesont-\nheijt, mijn soon presenteert sijn\ngehoorsamen dienst is Godt lof\nnoch gesondt, en ick blijve etc\n\nrecu 26e Dec: 1690\nDublijn den 19 Nov\n1690\nIck ben heden acht daegen in dese Stadt\ngekomen, omme verscheijdene affai-\nres met de heeren Hooge commissarisen\nvan dit Rijck t'adjusteren waer over\nwij dagelijcks besich sijn, en vermeijne\nbinnen twee daegen weder naer\nKilkennij te vertrecken. Uijt Enge-\nlandt hebben wij nu in drie weecken\ngeene brieven gehad vermits de\ncontinueele contrarie windt, en\nstorm, het welcke facteus is\n\nWant wij sijn, als vande rest vande\nwerelt afgesondert, en waerom ick\nhoop dat soo eens dit landt quite-\nren kan, daer uijt weder in te-\nkomen voor de rest soo ist goedt,\nen abondant van alles maer dat\nspreecken kan deucht daer niet.\nOnse Armee leijdt nu inde winter\nquartieren die slecht om de schaersheijt\nvan Huijsen sijn en wijdt is uijt\ngebreijdt, waerom wij nu en dan\ndoor de Rapparies geincommodeert\nwerden, de vijant houdt sich\nanders stil en sijnse seer geneces\nsiteert. hoopende noch al op een\nsecours van Vranckrijck, en soo se\nwaepenen, Amunitie en andere\nbehoeften kregen souden se ons\nnoch wel eene campagne ophouden,\nIck sach het bij de See gaern\nverhindert doch daer op werdt\nlangsaeme ordre gestelt. Men\nheeft mijn voor twee daegen\ngeadviseert dat een frans fregat\nint inkomen vande rivier de Shan-\nnon\n\nMet kat en muijs soude gebleven\nsijn en dat daerop sijn geweest\nveele amunitie, waepenen en\n28 officieren. Soo dit waer is, ist\neen teken dat men d'eijeren van die\nkant noch seconderen wil, ick\nben van opinie dat men se behoor\nde eene gaude brugge te maecken\ndoch daer wil men noch beswae-\nrelijck aen en men heeft in Enge-\nlandt te grooten sin in confisia-\ntien. Milord Sidneij die een vande\nHooge commissarisen is, en benen-\nfens d'andere mijn veel eere aen\ndoet\ndans le Conseill d'Etat sera ala\nville d'Utrecht, et celle de l'admi-\nrauté de Rotterdam à Messieurs\nles Nobles, ainsi il me tiendra\nqu'a Vostre Majeste d'ordonner quand\nl'un et l'autre reste, et mon grever\nqui si bien que moij n'embitione\nque de finier ces jours pour le\nservice de Vostre Majeste les en sera\nredevable avec un profund respect\net moij se les demande pardon\nsi pendant son absence j'ose le\nfaire c'este bien humble priere\n\nCopie de ce que j'ay escriet\naux Roy sur le subject de\nMonsieur d' Amerongen.\nAvant partir de Londres j'avois\nprise la liberté de faire une tres\nhumble priere à Vostre Majeste de la\npart de Monsieur d'Amerongen\nqui souhaite de continuer dans\nle Conseill d'Estat, apres ca com-\nmission expirée, ce que se poura\nsi Vostre Majeste a la bonte de l'ac-\ncorder sur le mesure prié que\nMonsieur vanden Cloese y'a continué\nle tour de la commission.\nHeer van Amerongen\nKilkennij den 26 Oct\n6 Nov. 1690\nrec a 11e. Dec:\nVoorleden donderdach weder hier geko-\nmen sijnde hebbe ick gevonden U.H.Ed:\nmissive vande 19e September, die soo nae 2\nMaenden oudt is, en waer uijt verne-\nme dat U.H.Ed: noch al is wachtende\nnaer positive ordres om de begonne\nne Tractaeten te voltrecken. Ick\nhoop dat se U wel.geb: inmiddels sullen\ntoegekomen sijn, en dat hij met be-\nnoegen dat werck sal ten eijnde\nbrengen t'sedert dat sijn Majesteit naer\nEngelandt vertrocken is, soo hebbe\ndaer ontrent niet verders kunnen\n\nDoen, oock niets daer van vernomen\ndan den Heere Hartoch van Wirtenberch\nsoo wel als Monsieur de la Forest en\nick wij wenschen van harten dat\nhet mach een goeden uijtslach heb-\nben Den goeden la Forest heeft\nvoor een weeck adrie seer aelendich\naent Graveel gelegen, doch een steent-\njen gelost hebbende is weder fris\nHij is geen sterck Man ick ben sijne\nMajesteit van denemercken met onder-\ndanich respect verplicht, dat aen\nsijn ootmoedigen dienaer soo\nGoedertierent gelieft te dencken\nen soude het mijn eene groote\ngenade wesen soo d'occasie heb-\nben konde, sijne handen te kussen.\nIck ben hier wat verre geenga-\ngeert doch hoope dat Godt ons\nhaest het geluck sal geven\neen kort eijnde te maecken\nen dan soo sal ick mijn groote eer\ngeven soo hier toe gelegentheijt\nvinden kan Mijne laeste brieven\nuijt Engelandt spreecken niet\nals vande groote concenten die\n\nHet parlement, soo voorde arrie-\nvages vande Armee, als d'aenstaende\ncampagne fraegen, en dat alle\nde caballes  in voordeel van den\nKoninck verandert sijn en een\nijder even seer hoopt, om sijn\nHof te maecken waer toe het\nglorieus eijndigen van onse Cam-\npagne met het nemen van\nCork en Kingsaijll niet weij-\nnich augmenteren ? sal, den\nHeere Hartoch van Wirtenberch\nbeneffens sijne verdere gene-\nraels\nHebben hier toe met haere troepes\nveel gecontribueert en eere inge-\nleijt, onse Ruijterije heeft het\nCanon, en d'aertellerie tien mijlen\nverre haelen moeten, ende halve Car-\ntouwen trecken, U.H.Ed: kan dencken\nhoese sijn geharasseert, en heeft\nsijne Majesteit redenen voor onse Ruij-\nterije gedurende de campagne ver-\ngenoegt geweest te sijn, ick sal\nvoor mijn Regiment al een tame-\nlijcke Recruijte van nooden hebben\nen den Heere Blanche par naesten\n\nMoeijelijck vallen, of hij mijn niet\nettelijcke wel gemaeckte oude\nRuijters, die gedienst hadden, van\nd'een en ander goedt vriendt\nweet over te krijgen nu manc-\nqueert mijn de tijdt en waer-\nom moet eijndigen, en blijven\netc.\nInt leger te Goulden Brits den 10e\nAugust:  St Nov: 1690.\nIck hebbe gisteren d'eere gehad t'ont-\nfangen van U.H.Ed: twee brieven den\neenen vande 18, en den anderen sonder\ndatum. Sijne Majesteit is voor vier dae-\ngen naer Doublijn en verder naer\nEngelandt vertrocken, doch  ick sal\nU.H.Ed: laesten brief aen den Grave van\nPordland toe senden, en versoecken\nover den inhoudt met sijn Majesteit\nte spreecken. Voor het vertreck\nvande koninck hebbe occasie gehad sijn\nMajesteit over U.H.Ed: voorige brieven t'on-\nderhouden die mijn seijde sijne senti-\nmenten aen den Raet pensionaris ge-\nschreven te hebben, oock soo hebbe\n\nMet den Heere Soutwel int lange en\nbreede gesproocken, en kan in confi-\ndentie mercken, dat men in den (----)\nMylord soo grooten vertrouwen\nniet heeft, als U.H.Ed. van hem gelieft\nte schrijven, waerop deselve gelieft\nte reflecteren, en sijne mesures naer\nte nemen, de redenen sijn mijn onbekent,\noock soo moet het niet blijcken, U. H.Ed.\nsal of aen den Heere Grave van Pordland\nof aen den Heere Hop somwijlen schrijven\nmoeten soo hij iets verders aen sijne\nMajesteit gelieft te hebben voorgebracht,\nwant ick vermits sijn vertreck te\nverre afben. De Koninck is alleen\ngeaccompagneert van sijne guardes\nTe paerdt den Grave van Schonberch,\nAuwerkercken, en Pordland met\nsijn verdere Hofstoet naer Enge-\nlandt vertrocken, laetende aen den\nGrave van Solms het commande-\nment over het leger, en aen mijn\nd'eer van te commanderen de Ruij-\nterije, met drie generaels majoors\nonder mijn, tee weeten de Heeren van\n's Gravemoer, S(ieu)r jan Lainier, en La forest\nBij het voet volck sijn den Hartoch\nvan Wirtenberch, en de generael\nHeer Douglas, de generael Majoors\nKerck, Tettou, en Graef van\nNassou, onse Armee is noch\n\nontrent de dartichch duijsent man\nsterck, en waer onder sijn seven duijsent\npaerden, den Vijant heeft, sich weder\nversamelt achter de Rivier De Sha-\nnon, en hebben t'sestich duijsent,\nSielen soo van vrouwen als kinde-\nren bij haer wij hebben door detache-\nmenten eenige fortressen en seehaven\ndoen wech nemen die sich alle sonder\nslach of stoot overgeven, en soo-\nhaest ons volck weder bij ons is hier\nweder op den Vijant toe marcheren\ndie ick hoop dat ons sullen wach-\nten soo wij maer den shanon pas-\nseren kunnen want wij doen niet\nals marcheren, en dat geeft veele\nsiecken. Wij vinden inde plaetsen\ndie sich overgeven groote Magasij-\nnen, en sijn al Meester van Wexfort\nWaterfort, donganon, en meer an-,\ndere seehavens Corck en kingseijl eijc-\nwachten oock maer, dat men haer\nsommeren laet, in somma den vij-\nandt heeft niet meer overich als\nde provintien van Coneck, Galluaij\nen dat achter Limmerick leijdt\nDen Heer van Middachten die noch\nfris is, en sijn onderdanich respect\npraesenteert is Majoor van mijn\nRegiment geworden. Gisteren hebben\nwij bij den Hartoch van Wirten-\nberch gegeten, en een halven roes\nop de gesontheijt vande deensche\nvrinden gedronken, ick blijve etc\n\nKilkennij den 7/17e december 1690\nU.H.Ed. missive vande 31e oct hebbe ick\nGisteren acht daegen ontfangen naer\ndat wij in eene geheele Maendt geene\nbrieven uijt Engelandt en noch veel\nminder uijt Hollandt hadden gehadt, dit\ngebeurt ons dickwils en is een vande\nfacieuste accidenten van dit Rijck\nja t'is bij Cromwels tijdt gebeurt\ndat se een mael elf a twaelf weecken\nsijn geweest sonder tijding uijt\nEngelandt te ontfangen Het geeft\noock al somwijlen groote verlegent-\nheijt, met de betaelinge vande Armee\nen hebben wij nu laest bij dese\n\ngedurige contrarie windt seer geem\nbrasseert geweest, sijne Majesteit char-\ngeert, mijn niet alleen met het\nCommandement maer oock met\nde Betalinge vande Armee, en hebbe\nick selver het begeven van alle\nde compagnien soo te paerdt als\nte voet en de mindere Charges\nuijt genomen vande deense troepes\nde welcke aen den Hartoch van\nwirtenberch gelaeten werden, Het\nis eene groote confidentie en veele\neer, maer oock eene onbegrijpelijc-\nke moijte, en men laet mijn\nnacht noch dach rust, waerom\ngedienstich bidden moet dat U.H.Ed. mijn\nsomwijlen gelieft te pardonneren, soo niet\naltoos even preciese op sijne brieven antwoor-\nden kan den Heere Hartoch van Wirtenberch\nheeft mijn d'eere gedaen met de Monsieur de La forest?\nen den Brigadier Ellenberch, vande week hier\ndrie a vier daegen geweest te sijn, en hebben\nwij niet gelaeten of wel den Heere Hartoch\nsober is, U.H.Ed. en alle goede vrienden in dene-\nmercken haere gesontheijt te drincken\nden Brigadier boven genoempt staedt\n\nBeneffens den commissaris jonck in korte daegen\nnaer denemarcken te verreijsen, en sal U.H.Ed.\nraport kunnen doen hoe wij met de wilde\nijren huijs houden, ick hoope dat gedachte\nBrigadier, eene goede Recruijte volgens mijn\nproject op onses konincks en sijne Majesteits\nvan denemarckens bejaegen voorde deense\ntroepen, in korten te rugge brengen sal\nWij hebben en hem en die Natie noodich\nwant het sijn braeve Soldaeten, en hier\nvan is dese somer al tamelijck de proef-\ngenomen, en wenste ick van har-\nten dat haer getal hier eens soo\ngroot waer, want het plaisier\nis sulcke te commanderen ick\nhoope dat U.H.Ed: nu haest met\nsijne Nogotiatie sal vaerdich sijn\nen dat d'Aliantie offencire en\ndefencie heeft mah geslooten\nwerden, want men met recht de\ncomparatie van man en vrouw\ntussen denemarcken, en Hollandt\nmaecken kan, die wel eens kijven\nmogen maer weder moeten vrien-\nden werden en haer beijde intrest\n\nIs eene nauwe Aliantie te\nhebbe, ick bidde vergiffenisse,\nsoo buijten mijne leest gae, mais\ndes souhaits du ceur la bouche\nparle ick hebbe vande weeck\neen elegante en groote brief in\ngoedertierene larmes van sijne\nMajesteit ontfangen en naer een\nlang antwoort het nevens\ngaende daer bij influeren laeten\ndat hoope U.H.Ed: sal bevallich\nsijn en een goedt effect hebben\nIck blijve verder met respect\nEtc.\nClomwell den 21/31dec 1690\nIck hebbe d'eere desen dach t'ontfangen\ntwee brieven vande U.H.Ed. van 15/25 en 18/28 nov\ndat deselve wat lange tussen wegen sijn,is\nniet vreempt. wij zijn hier op een Eijlandt\nen als weder en windt niet dienen wil\nsoo kant niet anders wesen. Niuws is\nvan hier niet veel te berichten, dan dat\nde vijant sich seer flatteert met frans\nsecours te sullen krijgen doch ick hoop\ndat het sal belet werden en dat men\nin tijdts een Esquadre Scheepen naer\nCorck en Kingsaijll senden sal waerom\nick bij alle posten aen houde, om den\nvijant bij de see te benauwen want\n\nWant wat wij te landt doen is niet\ndan onse troepes fatigeren, soo lange\nse de zee openhouden, en vervarsingen\nen toe voer uijt Vranckrijck krijgen\nals daegelijcks geschiedt, se sullen oock\naen geen manschap mancqueren want\ndie hebben se genoch, en alle de paepse\nijeren daer het landt vol van is, sijn\nvoor haer, ick derve het niet wel seggen\nmaer is al soo vreempt niet, want se\nvechten pro Aris et focis maer wat\nhaer mancqueren sal, dat sullen sijn\nKleedingen geweer officieren, Mun-\nnitie, mont kosten, en oock paerden\nhet laeste sal Vranckrijck niet senden\nmaer haer ontrent de rest niet ver-\nlegen laeten soo se ons hier door noch\neene campagne konden op houden,\nIck hoope van neen, want bent hier\nseer moede en wenste weder over\nzee te sijn. den Hartoch van Wirten-\nberch is oock van mijn sentiment, en\ndaerom doen wij wat practikabel is\nom t'werck te verkorten, en den\nVijant in te sluijten wij doen selver\nnu een moevement, naer de Shan-\nnon omse weder wat over te drijven\nen de fourages te benemen doch\nhet weder is soo quaet dat ons de\nSoldaeten jammeren en ick niet\n\nWeete of wij ons desseijn wel sullen\nkunnen voltrecken, ick hebbe aen\nBlanche versocht of hij van eenige\ngoede vrienden in denemarcken niet\neen vijfen twintich Ruijters te\nbekomen weet, doch sal der nu wel\ndartich noodich hebben, en derve U.H.Ed\nniet bidden of der geene apparentie is\ndaer toe van sijn majesteijt parmissie te\nverwerven het is mijn te doen om\nbraef volck te hebben die gedient\nhadden, en van aensien waeren Het doet\nmijn leet, dat U.H.Ed. bij mancquement\nvan ordres niet verder ontrent sijne\nTractaten avanceert, die ongetwijfelt\nsullen gedepescheert werden als de\nKoninck inde Haeg is, ick blijve etc.\n\n\nlande\nA monsieur\nMonsieur le baron de\nRheede Seigneur dÁme\nrongen\na Coppenhage\nrec 15e Dec\nKilkennij den 9/19 nov 1690\nNaer dat in lange het geluck niet\nhebbe gehad brieven van U.H.Ed.: t'ont-\nfangen soo sijnder mijn gisteren drie\nte gelijck behandicht, als vande 19\nsepte 3 oct, en 24 oct, het is seer\nmoeijelijck dat de posten soo-\nirregulier gaen, en dat men op\nhet bekomen van brieven soo-\nweijnich staet maecken kan\ndat bij het passeren vande see en\nde contrarie vrinden toe kompt\n\nDe slechte tijdingen uijt Ungeren\nen dat den Keijser een gedeelte van\nsijn Armee weder vande Rijn naer\nhet boven gemelte koninckrijck\ndoet marcheren, hebben wij giste-\nren oock verstaen, dit wil allent\nhalven eene groote verslaegent-,\nheijt geven en sullen de gealieer-\nden tegens het aenstaende jaer\nbetere mesures nemen moeten\nof Vranckrijck sal het spel mees-\nter werden men schrijft mijn\nMede van Londen dat Sweeden\nsijne troepes vande Rhijn getrocken\nheeft, doch op praetext dat Media-\nteur praetendeert te wesen, en door\nsijnen Minister bereets de media-\ntie aen onsen koninck heeft doen\naenbieden oock soo werdt mijn\ngeschreven dat den prins van\nWaldijck quiteren wil en niet\nmeer te velde gaen wie dan het\nleger van den staet commanderen\nsal wil den tijdt leeren\n\nWij sijn hier inde winter quartie-\nren doen weijnich dan nu en dan\naende Vijant de fourage te verbran-\nden, en de subsistentie te benemen\naen wie de levens middelen se-\nkerlijck schaers werden en soo\nmen haer het franse secours ver\nhinderen kan, gelijk ick hoope\ndat met weijnige Fregatten\nsal kunnen geschieden willen se\nvande Winter in slechten staet\ngeraecken, ick hebbe voor twee\ndaegen bij den Hartoch van Wirten-\nberch geweest, daer met Monsieur\nDe la Forest, en den generael\nMajor Tettou U.H.Ed. gesontheijt\ngedroncken hebbe den Heer van\ns'Gravemour die met mijn was\npresenteert sijnen dienst, en sloech\nmet Tettou een halve Bataille\ndoch ick was beschaempt den\nHollander te sien krengen ick\nstae over morgen naer dublijn-\nvoor acht a tien daegen te gaen\nom met de Heeren commissiona-\nrisen verscheijdene saecken\nte reguleren, al is d'Armee\n\ninde Winter quartieren soo vinde ick\nnoch al tamelijck mijn werck\nen d'eere van soo considerablen Com-\nmandement heeft oock sijn moij-\nte, ick blijve etc\nden Heere Blanche moet excuses\nversoecken soo mijn de tijdt ont-\nbreeckt om hem dit mael t'\nantwoorden\nrec a 28 jan-\nKilkennij den 15/25/ dec 1690\nU.H.Ed. missive vande 4/14 nov.is mijn bij de\nlaest gekomene post behandicht, waer\nuijt verneme dat U.H.Ed. eijndelijck een\ndefencive Tractaet geslooten heeft, ick\nwensche dat het offencive daer op mach\nvolgen op d'intressen en van denemarcken\nvan Engelandt, en onsen staet mogen\neen sijn en haere machten des te formida-\nbelder werden, Sweeden heeft sich inder\ndaet al veel bloot gegeven, en getoont\nde gealieerden weijnich toe gedaen\nte sijn en sal men tegens de Somer\nsien op wat grondt haere aengebodene\nheere van Amerongen\n\nMediatie leijdt. hier staen de saecken\nnoch als vooren, en d'ijeren blijven haer\nopiniatreren, en hoopen op Frans secours\nsoo van generaels als Amunitie\nen alderhande provisien, volck hebben\nse noch genoch want het gansche landt\nis voor haer, te weeten de papisten\nsoo dat se daer aen geen gebreck\nhebben sullen ick ben t' sedert ses\nweecken Solliciterende om een eijsfregatten\nontrent de Shannon te mogen hebben\nen het franse secours te verhinderen\ndoch kanse noch niet wel machtich\nwerden, en of men se al te landt\nBenauwt soo sien ick geene apparentie haer te\ndwingen dan bij forces soo lang alse het water oopen\nhouden, en bij de Winter kan men hier te lande\nweijnich beweging doen wij sullen niet te min\neersdaechs eens uijt trecken, omse weder ist\nmogelijck wat over de shannon te drijven. de Win-\nter begint hier fell en heeft t'sedert twee\ndaegen hier sterck gevroosen, doch men seijdt\ndat de forst hier te lande van geen duur is\nDen koninck sal nu apparent naer Hollandt\n\nvertrocken sijn of staet op sijn vertreck, ick hoop\ndat U.H.Ed. mijne laetste sal ontfangen hebben, en\ngesien wat ick sijne Majesteit bij missive hebbe\nversocht, opt subjecte van U.H.Ed: commissie inde\nRaet, de vrouwe van Amerongen heeft mijn geschre-\nven over de proosdije vande dom doch daer ontrent\nsoo vinde ick drie difficulteijten d'eene is het proces-\nmet diest, ten tweeden dat se maer 13 a 1400 gld-\ns'jaers, en geen Compagnie paerden waerdich is\nen ten darden dat niet weete voor wie de\nCompagnie geeijst werdt, het sij dat de parsoon\nGedient heeft of in staet van dienen\nis, het was anders geen quaet benifi-\ntie voor een jonger soon en waerdich\ndat men sijne gedachten daer ontrent\nliedt gaen wanneer daer toe apparen-\ntie was waermede verblijve etc\nHet ongeluck aen U.H.Ed: stal en koets\ninde Haeg overkomen doet mijn van\nharten leet, doch dit wil nu occasie\ngeven om het eene en andere niuw\nte maecken, ick hoop de stal wil\nnu wat naeder bij t'huijs komen\n\n\n\n\nrec 23e februarij\nClommell den 11 Jan 1691\nIck hebbe met den Heere Hartoch van\nWirtenberch eenige daegen op de marsch\ngeweest en die oorsaeck mijn d'eere\nniet geven kunnen t'sedert drie a vier\nposten aen U.H.Ed: te schrijven onse\nverrichtinge is geweest het ruineren\nvan een gedeelte van s'Vijants quar-\ntieren, inde Graefschap Limmerick, on-\nderttussen dat den generaell Majoor\nTettou, inde graefschap Kerrij is geweest\nen den Vijant voor hem vluchtende\nalle haere voederage hebben\nHeer van Amerongen\n\ngeruineert, soo dat hij eijndelijck geene\nsubsistentie vindende, en het quade\nweder hem overvallende, heeft moeten\nretireren en sich vergenoegen met s'vij-\nants quartieren te destrueren en is den\nvijant gemelt van alle kanten voor\nons over den Shannon geretireert\nse loopen soo wel, dat niemant haer\nin haelen kan ofse beter standt hou-\nden sullen wanneer haere franse generaels\ngekomen sijn wil den tijdt leeren\ntot noch toe is van dat secours\nniets te voorschijn gekomen\nAlhoewel daer van veel gerugt\nwerdt, onse Engelse troepes hebben\nden Vijant oock over al gepouceert\nen weder over de Shannon doen retire-\nren, sooden Vijant uijt vranckrijck\nsecours krijgt sal dat tot geen\nander oogmerck sijn, als om den oor-\nloch hier noch een jaer te doen\nduren en dan sullen dese Arme lieden\nals de Messinoisen gehandelt werden\nen ondertussen het gansche landt\ngeruineert, dat sommige beginnen te\nsien,en die alse kunnen souden willen\noverkomen waer toe ick hoope dat\n\nDe clementie vande Konink tende-\nren sal ondertussen dat men te gelijck\nalle prepareert om de campagne\nmet vigeur te beginnen, ick bidde\ndat Blanche mijn een letter antwoort\nof ick de Ruijters te verwachten hebbe\nen moet met haest eijndigen en\nblijven etc\nMeVrou Haisting suster van me\nVrou Molisword heeft dese daegen\nhier passerende bij mijn gegeten\nrec 27/17 februarij\nDublijn den 24 januarij 1691 3 feb\nErgisteren hier inde stadt komende, hebbe\nick gevonden twee brieven van U.H.Ed. vande\n12 des en 9e jan waer uijt met leetwesen\nsie dat het offencive Tractaet met sijne\nMajesteit van denemarcken noch niet naer\nwensch is geparfectioneert, Ick hoop dat\nde komste van sijn Majesteit van Engelandt\ninde Haeg de saecke verhaesten sal en\ntot een spoedich eijnde brengen, ick\nhebbe voor eenigen tijdt aende Heere Blanche\ncommssie gegeven of niet een 25 of 30 Ruij-\nters in denemarcken souden te bekomen\nsijn en krijge van hem geen antwoort\nhet welcke mijn seer verlegen maeckt\nHeer van Amerongen\n\nWant wij inde maendt van Maert moeten\ncompleet sijn, en d'Equipages sal ick\nwel bij de handt krijgen, maer de man-\nschap sal mijn mancqueren waerom\ngedienstich versoecke dat den Heere Blan-\nche hier toe noch een tentamen\ndoet en mijn bescheijdt doet weeten\nsoo dese Ruijters te bekomen sijn, konde\nmen daer bij een corporaell of wacht-\nmeester maecken, en deselve direct\nop Londen aende Tresorier vander Esch\naddresseren die se verder sal sen kunnen\nherwaerts senden, Monsieur de Tercon-\nnell is te Limmerick aengekomen\ndoch heeft, soo veel als tot noch\ntoe hooren kan geen Soldaeten noch\nofficieren met gebracht, de tijdt\nwil leeren wat secours datter ver-\nder volgen sal. bij aldien het niet\nconsiderabel is, sal men met dese\ncampagne het werck kort maecken\nin philips Toun sijn maer weijnich\nijerse Cabbens gebrant, en wij hebben\nnoch guarnisoen int Huys ick\nblijve met haest etc\n\n\n\nrec 19e marti\nDublijn den 4e/14e feb 1691\nIck hebbe d'eere gehad gisteren t'ont-\nfangen U.H.Ed.: missive vande 16 jan, t'sedert\nacht a tien daegen is de windt altoos\ncontrarie geweest soo datter geene brie-\nven van hier naer Engelandt noch\nHollandt hebben kunnen overgaen\nen waerom eene te rughoude die\nvan weijnich belang was, en voor drie\ndaegen geschreven hadde, U.H.Ed: heeft\nuijt mijne voorige gelieven te sien\ndat de Graef van Terconnell uijt\nVranckrijck bij d' ijeren is gearrireert\n\nDoch soo veel wij tot noch toe hebben\nbespeuren kunnen, heeft hij weijnich\nsecours, en troost met gebracht\nsoo dat ick hoop d'ijeren sullen sich\nnoch bedencken, en tot submissie komen\nsoo niet wil dit Rijck een misera-\nbel sacrifitie voor Vranckrijck sijn.\nIck hoop dat den koninck nu inden\nHaeg sijnde veele saecken daer sullen\nwerden geadjusteert, insonderheijt de\nverdere Aliantie met denemarcken\nwaeraen van beijde sijden seer veel\ngelegen is, de vrouwe van\nGinckel schrijft mijn dat se met\nde Vrouw van Amerongen inden\nHaeg is, daer nu veel te doen sal\nsijn de koninck heeft soo wij hoo-\nren, al groot perijckel opde zee uijt gestaen\nDen Hartoch van Wirtenberch is\neenige daegen niet wel geweest doch\nsoo mijn gisteren geschreven heeft we-\nder beter, ick geloove het half van\nchagrijn is, dat hij hoort de difficulteij-\nten, die gemoveert werden, om recruij-\ntes uijt denemarcken voorde Deen-\nse troepen hier sijnde se krijgen,\nhet soude groot jammer sijn, dat\neenige wierden gereformeert, of\n\ndat se van tijdt tot tijdt met\ngeene recruijtes van haere eijgene\nNatie onderhouden wierden, ick derve\nU.H.Ed. niet vraegen of dit niet behoor-\nde aen sijne Majesteit van Denemarcken\nniet een weijnich te werden geremon-\nstreert dewijl dese troepes die sijn\nnaem draegen bereets soo veel reputatie\nhebben geacquireert, en hoe beter de\nselve werden gemaintineert het niet\ndan voor sijne glorie stercken kan.\nIck hoop dat Blanche mijn aengaende\nmijne Ruijters antwoorden sal\nen soude seer verlegen sijn soo se\nniet te bekomen waeren, ick blijve etc.\nHr. van Amerongen\nDublijn den 15/\n25 feb1691\nrec: 19e martii\nIck hebbe d'eere gehad gisteren twee\nbrieven van U.H.Ed. t'ontfangen vande\n23, en 31 jan: de proosdije van den\ndom belangende, hebbe ick aende vrou-\nvan Amerongen oock geschreven, dat\nop soo ridicule conditien daer in geen\nsin, en hadde, en gelijck U welgeb(orene)\nwel gelieft te seggen wij moeten geen\nprocessen koopen. Godt hoope ick\nsal mijne familie elders segenen.\nHet doet mijn leet, dat U.H.Ed.\nmijn brief niet heeft ontfangen\n\nWaer bij gesonden hebbe copije vant geen\naende koninck opt subject van U.H.Ed\ncommissie hebbe versocht, en ick\nhoope dat het nu in den Haeg effect\nhebben sal, ick beduchte oock dat\nmijne brieven niet overgekomen\nsijn waer bij Blanche hebbe ver-\nsocht, vier en twintich Ruijters\nbij d'een en ander goede vriendt\nvande deense troepen te procureren\nen soude ick seer verlegen sijn soo\nhij mijn deselve niet wist te\nschaffen want den Hartoch van\nWirtenberch meijnde dat se niet\nsouden geweijgert werden en waerom\nbidde bij aldiense noch te bekomen\nsijn, dat te mogen directe op londen\naende commissaris vander Esch werden\ngeaddresseert, U.H.Ed. gelieft te schrij-\nven datter hoop is voorde Recruijtes\ntot de deense troepes waer over\nden Hartoch van Wirtenberch seer\nverblijdt sal sijn en hebbe ick\nhem gisteravont Uwelgeboren missive\ngesonden, men schrijft uijt Hollant\ndat de koningen van denemarcken\n\nen Engelandt, sich ergens ter halver\nwech ontmoeten souden doch dewijl\nU.H.Ed. hier van niet schrijft soo stelle\ndaer in geen geloof, Terconnell is\nuijt Vranckrijck gekomen, en heeft\nweijnich assistentie met gebracht\neenige kleijne provisien voor\nRekening van particulieren, en\nanders niet, d'ijeren laeten niet\nop een grooter secours te hoopen\nen blijven daerom opiniate\nHet is een fraij present dat mijn\nsoon van sijne Majesteit van Denemar-\nken\ngekregen heeft ick hoop dat hij die\nKonincklijcke gunst meriteren sal,\nen U.H.Ed. in allen deelen benoegen\nen satisfactie geven, ick blijve.\netc\n\nDublijn de 21/31 Maij 1691\nU.H.Ed. missive van 4 Maij is mijn met\nde post van gisteren terhandt gekomen\nen bidde ick vergiffenisse soo met de\ntwee voorige posten een weijnich in haest\ngeschreven hebbe vermits ick mijn soo\nveel spoedich als mogelijck is, om te velt\nte gaen, en d'Armee in ordre te brengen\nsoo hebbe soo veel werk dat van smor-\ngens vroeg tot s'avonts laet besich\nben en onse troepen beginnen uijt\nde Winter quartieren te marcheren\nen vermeijne ick oock binnen een\ndach a ses naer het rendevous te\nHeer van Amerongen\n\nvertrecken. Godt hoope ick sal ons geluck\nen sijn segen geven op dat wij dit Rijck\nhaest tot gehoorsaemheijt brengen\nmogen waer in mijn geen arbeijdt sal\nverdrieten laeten maer hadde het fran-\nse secours belet geweest, wij hadden\nhet werck beter koop gehad, en was\nde Vijant in sulcken staet dat se het\nonmogelijck geen ses weecken meer kon-\nden uijt houden, ick hebbe het genoch\ngeschreven, en de gansche Winter gebeden\nmen soude een esquader scheepen\nopde kust senden. Corck en King-\nsaijll sijn de beste Havens vande werelt\nDaer se hadden kunnen secuur we-\nsen, nu is den Vijant van alles voor\nsien, en die pro Aris et focis vecht,\ndie doet ordinaris sijn best, Mons.r\nde St. Rut sal haer generaell sijn,\nen is soo ick hoor een officier van lange\ndienst, hardij, maer wat brutaell\nen een van diegeene soo de meeste\nparsecutie in Vranckrijck aende protes-\ntanten heeft gedaen. Den Lt. Generael\ns'Gravemour en de generaels Majoors\nMacquaij, en Talmouts verwachte ick\ndagelijcks, de generael Majoor Ruvignu,\nis al gearriveert, ick sal int faveur\n\nvan Blanche schrijven doch beduchte\ndat hem twee saecken obsteren sullen,\nd'eene dat hij de voorledene campag-\nne niet heeft gedient en sich int hee-\nte vande oorloch absenteert, en ten twee-\nden dat de prins van Waldijck het\nRegiment voor een van sijne favoriten\nbegeert. Mijn soon is te Kilkennij mijn\nRegiment verzamelt sich met den Har-\ntoch van Wirtenberch soo dat wij ons\nnoch in veertien daegen niet sien sullen,\nhij is anders Godt danck gesont. Wij\nhebben hier Ruijters gekregen soo dat\nter nu geen sullen noodich hebben\nick blijve etc\nmijn soon is weder t'eenemael van\nsijn sieckte harstelt, en bij mijn hier\nint leger, hij is van verren weder\ngekomen, en heeft een swaere kranck-\nheijt uijt gestaen, Godt hoope ick dat\nhem verder bewaeren sal. Mijn soon\nvan Harvelt moet mijn excuseren\ndat hem dese post niet schrijve\nde tijdt mancqueert mijn, en sal mijn\nmet de naeste post oock d'eene geven\nt'antwoorden aen Monsieur de Falaiseur\n\nom tien uren de Revier passeerden\nom het Guee te sonderen en tot aen\ndes vijants wercken naederden soo datse\nde handen aende pallisaden sloegen in\nmijn gesicht, de vijant schoot sterck-\nen twee vande selve wierden gequetst\ndoch sult beijde eschapperen soo ick hoop\nIck hebbe haer voor desen dienst wel\ngeloont, en een vande drie dat een edel\nman was, d'eerste Hollantse standaer\nversproocken, die vacant komen sal\nten waere dat den Hartoch mijn\nprevenieert, en hem behouden wil.\nIck blijve etc\nrec : 1/11junij\nDublijn den 13 May\n1691\nDen Hartoch van Wirtenberch\ndie drie a vier daegen hier is geweest\nom alles wegens onse marsch met\nmijn t'overleggen geeft mijn den\nnevengaenden brief, en versoekt\ndat se secuur mach werden geaddreseert\nwij sijnt te samen in alles eens\nen ick arbeijde nacht en dach om\nd'Armee, ten spoedichste int velt\nte brengen, alsoude hieren daer noch\nwat mancqueren want tot noch\nHeer van Amerongen\n\nToe is het franse secours niet gear-\nriveert en den vijant in slechten staet.\nde post vertreckt, ick hebbe voor\ndit mael geen tijdt meer blijve etc.\n\nrec: 1 junij\nDublijn den 4 April/14 may 1691\nU welgeb heeft gelieven te sien uijt mijn\nvoorige dat tegens mijne verwachting\nvande Koninck ordre bekomen hebben, om\nde campagne hier te doen, en sijne Majesteit\nArmeen in dit Rijck te blijven commande-\nren. ick ben nu continueel besich om\nalles in staet te brengen, en hoop binnen\ntien a twaelf daegen de troepes te versa-\nmelen, sal U.H. Ed. met den eersten een\nlijste toe senden hoe sterck sijn Majesteits\nArmee in dit rijck wesen sal en wat\nGeneraels deselve salllen Commanderen\ns'Gravemour schrijft mijn datter eenige\nHeer van Amerongen\n\nverandering in vallen wil en is dit de\nreden waerom ick se noch niet hebbe\ndoen afschrijven. Het franse secours\nis noch niet gearriveert, en den Vijant\nis in quaeden staet, onse partijen slaen\nhaer overal, de meeste van haer Armee\nbegeven sich tot Rapparies, soo dat\nbij aldien het gemelte secours door\neen detachement op de kust te sen-\nden, konde verhindert werden soo\nsouden wij naer mijne opinie hier\nhaest kunnen gedaen hebben. Het welcke\nwij dagelijcks bij Expressen, naer\nEngelandt overschrijven, en represen-\nteren\nmaer krijgen weijnich antwoort\nGodt weet waer aen het hapert,\nsoo der om de Ruijters te bekomen\ngeen goede commoditeijt is, sal ick\nmijn behelpen soo veel als ick kan,\nen hebbe hier noch ettelijcke gekre-\ngen waermede blijve etc\n\nDublijn den 25de April 1691\n5de May\nick hebbe gisteren een onverwachte\ntijding gekregen dewelcke is des Ko-\nnincks bevel dat ick hier moet blij-\nven om sijne Armees te Commande-\nren, het welcke uijt mijn gissing\ngaet en soude ick liever in minder\ncharacter, en met weijnige eer\nin Brabant, en ontrent mijn\nVaderlant sijn geemploijeert ge-\nweest doch ick moet patientie\nhebben en dere het niet refuse-\nren, d' Armee sal buijten de\ngarnisoenen uijt ontrent die\n\nM/\nM/30 man bestaen en daeren boven\nsullen wij noch M/15 landtmilitie\nwaepenen, ick sal UwelGeb: daer van\nde lijste par naesten toe senden.\nDen secretaris Neve werd door den Hartoch\nvan Wirtenberch aen mijn geaddresseert\ngeweest, hadde ick vande middach\nten eeten versocht, doch was soo\nhaestich, dat juijst afvoer, soo\nals wij aen Tafel souden gaen\nIck hoop hij sal een beter rapport\ndoen, als die soo qualijck vande betalin-\nge gesproocken hebben, ick hebbe\nden ganschen dach en halve nacht\ngeschreven, moet vergiffenisse\nbidden, soo desen niet langer maec-\nke en blijve etc. Wij verlan-\ngen seer naer de Recruijtes, en\nt' sal mijn aengenaem sijn soo\nnoch Ruijters krijgen kan, ick\nhebbe der oock uijt pruijssen\nontboden doch se komen niet\nt'onpasse\n\n\n\n\n\n\nDublijn den 2/3e Maij 1691\nMet de laeste post hebbe ick d'eere\nniet gehad brieven van U.H.Ed. t'ontfangen\nonse Armee begint sich te bewegen\nen heden is de rest van onse Artellerie\nvan hier gemarcheert ick vermeijne\nbinnen twee daegen te volgen in aen-\nstaende weeck sullen wij beginnen t'age-\nren, uijt de nevengaende lijst gelieft U.H.Ed.\nte sien, wat Armee wij te samen brengen\nwaer van ontrent tussen de tien en\nelf Bataillons sullen werden in Guar-\nnisoenen gelaeten, ick blijve met haest\netc\nde vijant sel stercker int veltkomen\ndan wij nu se ge nree ? gekregen hebben\nHeer van Amerongen\n\nrec: a. 1/11 May\nKilkennij den 13/23 April 1691\nT'sedert twee posten sijn geene brieven\nuijt Hollandt aengekomen, en hebbe\nick mijn d'eere gegeven op U.H.ed. laeste\nt'antwoorden, ick sal hier nu alleen\nbijvoegen een kleijne memorie vant\ngelt dat aen de deenen is gedistrilueert\nhier te lande t'sedert dat ick d'eere\nhebbe gehad van sijn Majesteit. Armee\nte commanderen, en de betaelinge\nvande selve te ordonneren waer uijt\nblijcken sal de valsche en inparti-\nnente rapporten bij La Gulle ? die\nick weijnich hebbe gekent, en\nHeer van Amerongen\n\nonder de Deense natie voor een\ndebauchant en misserabel wedde\ngedenieert gedaen heeft, en heeft\nden Hartoch van Wirtenberch hier\nvan een andere getuijgenisse gegeven,\nsoo de selve mijn versekert heeft\nmaer ick beduchte en remarqui-\nre met reden, dat alle sijne brie-\nven voor een gedeelte werden\ngeseponeert, en ter ooren van\nden koninck van denemarcken\nniet komen apparent door soo-\ndanige Ministers, die haere Reke-\nninge bij Vrankrijck vinden\nen desen eerlijcken heer soecken\nverdacht te maecken ofse te gelijck\nsal den tijdt leeren Wij verlan-\ngen niet inpatientie naer den uijt-\nslach van Mans, doch de vreese\nhaer Meesters intrest waernemen\nis grooter dan de hoop, ick hebbe\naen sijn Majesteit een nieuwe voor-\nslach aengaende de deense Recruijte-\nringe gedaen, en het is beter desel-\nve in vreemde landen te soecken,\ndan soo goede corps te sien onder\nsteecken en ick hoope en twijfele\nniet of sal noch t'avont of morgen\n\nbij den koninck van Denemarcken\ngenade vinden voor den dienst die\nhier in soeck te doen gelijck ick altoos\nhebbe getracht sijne troepes soo\nveel faveur te bewijsen, als mijn\nis mogelijck geweest vorders soo is\nvan hier weijnich niuws voor dit\nmaell de schrijven de ijeren verwaech-\nten noch haer groote secours, en\nmen begint te seggen dat ick vande\nsomer in Hollandt dienen sal, waer\nnaer verlange, ick blijve etc\nrec a 14e meij\nKilkennij den 5/2 april 1691\nIck hebbe d'eere met de post gister\navont gearriveert t'ontfangen U.H.Ed\nmissive vande 27e Maert die mijn\nvernemen doet dat de saecken continu-\neren in Denemarcken niet wel te\ngaen, dat den Hartoch van Wirten-\nberch en mijn leet doet Het is\nons echter lief dat Monsieur de la\nBarre andere tael heeft gevoert\ndan eenen droncken Monsieur de la\nBalle die hier bij de deenen van\nweijnich merites is geoordeelt\n\nen of wel ick met dese post\nU.H.Ed. noch niet toe senden kan\nden staet vant gelt soo de deenen\ngedurende dese Winter en mijne\ndirectie ontfangen hebben, soo\nsal het niet te min par naesten\ndoen, den Hartoch van Wirten-\nberch is weder ten eenenmael\ngereconvalesceert, en een braef\nHeer het waer te wenschen dat\nalle de Ministers in denemarcken\nsoo waeren onse twee koningen\nsouden het haest eens sijn\nvorders passeert hier nu weij-\nnich niuws het franse secours\nnoch haere generaels sijn noch niet\naengekomen wij verwachten\nmet impatientie den uijt slach-,\nvan Mons. Godt geve aen onsen\nKoning de victorie en ick blijve.\netc\n\nrec a 4/14 meij\nKilkennij den 1/11e April  1691\nUwegeb. missive vande 10/20e Maert is mijn\ngisteren ter handt gekomen het\ndoet mijn leet dat de saecken in dene\nmarcken ten besten vande gemeijne\nsaeck soo slecht uijt vallen, het\nis wel een weijnich de traegheijt\nvan ons Hof t'imputeren maer\nmen behoorde daer oock wijser te\nsijn, die klachten die sekeren\nMonsieur de la Vale aent deense Hof\nover de betaelinge vande troepes\nheeft gedaen, sijn niet alle naer\n\nde Waerheijt en ick sal U.H.Ed. expres-\nse per naesten een staet oversenden\nvande betaelinige die daer is gedaen\nt'sedert dat ick d'eere hebbe d'Armee\nte commanderen en ick hebbe haer\naltoos naer proportie meer doen\ngeven als aende Hollanders het is\nwaer d'Armee is niet regulier\nbetaelt en ick ben selver elf volle\nMaenden Tractement ten achte-\nren niet tegen staende dat aen\nalle de generaels hebbe betaelinge\ndoen, doen wij verlangen seer\nWat tijding vande Belegering van\nBergen komen sal aen welckers\nontset veel dependeren wil soot\ngetenteert werdt, in den Haeg is\ntamelijck koeck uijt gedeijlt, doch\nick ben tot noch toe vergeten, den\nstaet is geluckich nu soo veel gene-\nraels te hebben het oude spreeck\nwoort seijdt dat veele officieren\nvechten scherp, ende vrienden sul-\nlen nu licht al datelijck aent\nwerck gekomen sijn, hier hebben\nwij weijnich niuws de franse gene-\nraels\n\nsijn noch niet gearriveert, doch\nd'ijeren voeden sich met groote\nhoop en sullen noch eene\nnumbrense Armee te velde\nbrengen, bij de vrouwen van\nAmerongen en Ginckel is een kleij-\nnen alarm geweest, men hadde\nuijt gestroijt dat ick gequetst wae-\nre doch t'is Godt Lof soo qualijck\nniet af geloopen, Ick hoop noch\nal dat vande somer sal mogen naer\nHuijs komen en blijve etc\nrec a 11/21 May\nDublijn den 29 April 3e Maij 1691\nIck bidde U.H.Ed om vergiffenisse,\nsoo mijn dit mael d'eere geve alleen\nin grooten haest tot antwoort op\nU.H.Ed. missive te seggen, dat verblijdt\nben de saecken in denemarcken sich\nweder beginnen te schicken, ick hoop\ndat alles noch sal den besten uijt\nvallen, en datter met die Croon een\nvolle Aliantie sal kunnen geslooten\nwerden, Naer de Recruijtes werdt hier\nseer verlangt, ick hebbe eenige dae-\ngen geleden aen U.H.Ed. een staet\ngesonden vant gelt dat dese winter-\nHeer van Amerongen\n\nBende/Aande deenen in gelt van tijdt tot\ntijdt is uijt getelt, boven vrij quart-\ntier en onderhoudt in eeten en drin-\nken soo datse inder daet soo groote\nRedenen van klaegen niet hebben, ick\nsal aen den Hartoch van Wirtenberch\nschrijven en proponeren of sijne\nbrieven voortaen in mijn pacquet\ngelieft te doen doch soo ick vertrecke\nals uijt Engelandt geschreven wedt\nen waer van alle uur de sekerheijt\nstae te hooren soo salt van geen\nlangen dienst wesen. Het franse\nsecours is tot noch toe niet gear-\nriveert, en ick hoop dat onse vlooten\nhaest sullen in zee sijn vont ver-\nder te beletten als wanneer de\nsaecken hier haest een eijnde nemen\nsal, want tot noch toe sijn de\nijeren seer g'incommodeert, ick blijn\netc\n\n\n\n\n\nDublijn den 5e/15e Maert  1691\nIck hebbe eenige daegen te Molingaer, en\ndaer ontrent geweest om op verscheijdene\nsaecken ordre te stellen en onder andere\nden Vijant oock eene kleijne visite gedaen\ndie met drie a vieren twentich hondert Man\ntussen Molingaer en atthloon over de Shannon\ngekomen waeren wij ontmoeten haer heden\nacht daegen geleden tussen sekere plaets Streemps-\ntauw en Athloon genaempt, se waeren avanta-\nsieus geposteert, en maeckte mine van ons af te\nwachten, doch op ons aennaederen soo quiteer-\nden sij d'eene pas voor en d'andere naer, en als\nwij haer eijndelijck een weijnich begonden te\npresseren verliet haer voetvolck haer aen-\nstonts en vluchten inde Mourassen daer se\nvande onse dede volgen doch d'ijeren sijn\n\nBeter te voet dan d'Engelse, soo dat daer\nniet boven de dartich a veertich vande ijeren wier-\nden gecappotteert, doch se verlooren soo-\nick vertrouw veel geweer, haere Ruijterije\nen dragonders welckers getal veel stercker dan\nhet mijne waer, retireerden sich naer Athloon\nen ponceerden wij deselve seven ijerse Mijlen\nverre, tot op een kleijne Mijl van die plaets\nveele en meest alle haere qualijck beredene\nraeckten Capot, want d'Engelse geven weijnich\nquartier aende ijeren en men heeft veele paer-\nden van haer gekregen die echter seer slecht,\nen qualijck onderhouden waeren en t'is remer\nkabel dat de Vijant niet een maell tegens\nons hebben gewent niet tegens staende dat\nwij seer weijnich en maer t'sestich dragon-\nders, en sij drie Regimenten dragonders\nbij haer hadden, en de gansche wech die\nWij haer pouceerden niet dan seer Vijlaijne\ndefilees, Bossen en steene straet waegen\ndat is vande Mot a Gronauw tot Athloon ick\nnoem dese plaetsen om dat den Envoijé Moles-\nword deselve apparent kennen sal dewijlse\nniet verre van Philipstauw  leggen, en\nick vertrouwe niet, dat de ijeren ons licht\nweder soo nae sullen komen laeten bij aldien\nse geen beter sin tot vechten hebben, in dese\noccasie hebbe ick niet boven de 300 soo hollant-\nse als franse Ruijterije bij mijn gehad, derest\nwaeren Engelse, gelijck oock een Engels gene-\nraell Majoor Sir Jan Lanier genaempt die\nveel vigeur toonde drie van U.H.Ed. brie-\nven sijn mijn gedurende dit uijt sijn behandicht\nals vande 23 januari, 8 en 13 feberwari, die mijn\nt'elckens met blijdtschap vernemen doen\n\nDe continuatie van U.H.Ed. gesontheijt Godt\nlaete het lange duren Wat Terconnell\nheeft met gebracht hebbe U.H.Ed: voor desen\ngeschreven, en dat het van weijnich belang\nis, doch men gerucht datter een grooter\nsecours staet te volgen twelcke ick hoop\ndat sal verhindert werden en hebbe de\ngansche Winter gebeden men soude een Esqua-\nder scheepen opde ijerse kust te Korck, en\nKingsaijll senden doch hebbe het tot noch\ntoe niet verwerven kunnen en waeren se\ndaer geweest ick vertrouwe dat IJerlandt\nwas gereduceert nu salder noch veel Christen\nbloedt vergooten werden en het landt verarmpt\nwant de Natien verbitteren seer op den\nanderen en ick voorsie dat d'ijeren sullen\nwerden geexterpeert sosijt niet haest\nop geven den Hartoch van Wirtenberch\nDie gevaerlijck sieck is geweest doch nu\nweder bij nae harstelt, en ick mij verlangen\nseer ofter geen Recruijtes uijt Denemarc-\nken komen sullen het is meer dan drie Maen-\nden geleden dat ick aen Blanche geschreven\nhadde of daer geen vierentwintich goede\nen wel gemaeckte Ruijters voor mijn Regi\nment te bekomen waaeren den Hartoch\nvande Wirtenberch meijnde datter wel een\ngoedt vrient of ettelijck soude te vinden\nsijn diese met parmissie vande Koning sou-\nde over laeten ick eijsche niet dan de bloote\nManschap en sal se van't Hooft tot de\nvoeten munteren doch gelijck ick geen\nantwoort krijge soo vreese dat mijne brie-\nven sijn vermist, en dit maeckt mijn\nseer verlegen, den Brigadier Ellenberch\n\nis nu apparent te Coppenhagen al voor\nlang aengekomen, die tot het bekomen\nen overbrengen van dese Ruijters goede aen-\nslach geven soude, ick versoeck datter een\nsmit of twee en een Sadelaer mach onder-\nsijn, ick sal mijn nu verder d'eere geven\nU.H.Ed t'onderhouden van twee domestique\nsaecken het eene is aengaende de proosdije\ndaer de Vrouwe van Amerongen schrijft\ndat Eckstet noch op aen houdt, en dat hij ten\ndeele van sijne ongerijmpde propositien\ndesisteert, met eene compagnie paerden soude\nte vreeden sijn en dat hij de 8000 gulden bij\ntarmijnen wilde afleggen, en alle jaer 2000 gulden\nvan sijne gages laeten staen, doch hier in\nvinde oock noch al swaericheijt voor eerst\nbij dese tijdt een man te recommanderen\nDie noijt niet heeft gedient, en apparent\nniet bequaem sal sijn en ten anderen dit\nCapitaell met 2000 op sijne compagnie te\ntrecken is seer wilt, want hij kan werden\ndoot geschooten, en soo lang hij leeft en den\noorloch duurt soo heeft hij sijne gages\nwel van doen soo dat dit niet wel aen\nkan gaen, de tweede saeck is dat de vrouw\nvan Amerongen mijn van een huwelijk\ngeschreven heeft van mijn tweede soon\nmet den Haegsen Cuirni?? over de den\napparent soo sal U.H.Ed. hier oock van\nkennisse gekregen hebben doch gelijck\nick noch hij parsoons noch sijn huijmenens\nnoch pan Goedt niet kenne, noch oock\nniet weete wat inclinatie min dechten\nvoor Redve heeft, soo hebbe hier op geen\n\nAntwoort geven kunnen dan dat hier\nvan eerst beter most wesen geinformeert\nen of het niet met beleeftheijt konde\nwerden gedelaijeert tot dat ick t'huijs qua-\nme onck dat men U.H.Ed. goedtvinden, en\nadries eerst weeten most, en ick hebbe\naen den koning parmissie gevraecht,\nom voor de somer te mogen Repatrieren\ndat hoope mijn sal werden toe gestaen\nwant ick naer Huijs verlange, ick hebbe\noock geeijst het Gouvernement van Breda,\nin cas dat Mijlord Pordland t'selve niet te\nhebben staet, en verlange wat antwoort\ndat ick opt een en andere krijgen sal, en\nversoecke dat U.H.Ed: hier van niet gelieft\nte doen blijcken, ick blijve etc en Heer van\nCralingen die dese middach met mijn eet pre-\nsenteert sijn dienst, en sal U.H.Ed gesontheijt\ndrincken etc\nrec 16e meij\nKilkennij den 20/30e maert 1691\nT'sedert mijnen laesten vande 18e is hier\nniet sonderlings voorgevallen, als dat\nmen op verscheijdene plaetsen den\nvijants partijen noch geslaegen heeft\nniet tegenstaende soo blijven de IJeren\nopiniatreren, en flatteren sich met\nhet franse secours, Wij prepareren\nons tot de campagne, en ick sal verlan-\ngen wat antwoort dat ick voor mijn\nparticulier van't Hof krijgen sal\nden oorloch sal hier bloedich werden\nniet soo seer ten opsicht van Vij-\nants\nHeer van Amerongen\n\nBestendige Armee als van haere Rappa-\nries, die seer aengroeijen al hoewel\nwij deselve geen quartier geven\nen sijnder binnen de drie weecken\nmeer dan vierhondert doot geslaegen\nen op gehangen, het sijn schelmen die\nnoijt vechten, daegs met ons volck\nom gaen en snachs breecken se haer\nden hals en se geven oorsaeck dat\nmen d'ijerse natie bij nae niet meer\ntrouwen kan, en sal exterperen moe\nten, dit is voor dit mael het geene\nwaermede Uwelgeb voor dit mael\nT 'onderhouden hebbe, en blijve etc\n\nrec 27e Ap(ril)\nKillkennij den 14/24 Maert  1691\nU.H.Ed. missive vande 20e feb is mijn voor vier\na vijf daegen behandicht, ick bidde vergiffe-\nnisse soo met ijder post niet schrijve doch\nick mancquere selden eenmaell ter weeck\nten naere dat hier of daer op reijs ben\nmijne affaires sijn swaer en werden daege\nlijcks swaerder, door dien de campagne\nop handen schiet en ick werde met alles\ngechargeert te meer nu de koninck in\nHollandt is, en in Engelandt weijnige\nsijn die het werck verstaen, ick hebbe\nparmissie gevraecht om te mogen\nHeer van Amerongen\n\nRepatrieren, doch weete niet oft mijn sal\nwerden toe gestaen, dese post is wel hono-\nrabel, maer seer painibel, en se salt\nnoch meer werden dewijl de koning vande\nSomer in Vlaenderen en soo verre van\nhier sal sijn de ijeren blijven opinatie\nen willen soot schijndt van Vranckrijck\ndependeren die haer handelen sullen\nalse met de Mennoisen hebben gedaen\nen bij haer werdt uijt vranckrijck\nniet verwacht, als drie a vier generaels\nparsoonen gelt, en amunitie, het\nwaer te wenschen dat wij eenige\nscheepen op de kust hadden om haer\ndit secours te beletten. Uijt de provintie\nbeginnen veele Lieden over te komen\nen protecie te versoecken, en staender\nsoo  gerugt werdt meer te volgen, die\nde france Tijrannije niet willen onder-\nworpen sijn, ick ben gisteren weder hier\ngekomen alwaer den Heere Hartoch van\nWirtenberch vondt, met de Generaels\nMajoors Tettou en La forest die mijn\nvier daegen hadden ingewacht, gedachte\nHartoch is Godt danck weder gesondt\ndoch haest al seer sieck ja buijten\nhoop van leven geweest. Hij is seer\n\nverlegen datter geen deense Recruijtes\nkomen en t'sal schade wesen twee\nRegimenten onder te steecken, ick\nvinde mijn oock seer geembrasseert dat\nBlanche schrijft ick de 24 Ruijters\nniet hebben sal, ze maeckten mijn\nde saecke soo licht soude die anders\nniet gevraegt hebben de Rapparies\nhebben t'sedert mijn absentie in mijn Regi\nment groote schaede gedaen, en bij nacht twin-\ntich paerden van Moersbergen in een\nstal verbrandt, dit Canaille quelt ons\nmeer dande vijant, en het landt is soo\nmiserabel van Huijsen voorsien dat men\nbij nae geene secure quartieren vinden\nkan, ick blijve etc\nrec: nrAp (ril)\nKilkennij d'en 24 Maert\n4 April 1691\nvoor twee daegen hebbe ick ontfangen\ntwee brieven van U.H.Ed. van 6 en 13\nMartij, die mijn met leetwesen doen\nsien, dat de saecken in denemarcken\nniet beter werden noch tot geen goedt\neijnde komen Bij aldien men de vreem-\nde troepens uijt dit Rijck treckt\nsalt haest weder tot sijn oude\nMeester geraecken ick segge dit\nniet dan dat lust hebbe hier te blij-\nven, want ick hoop dat voor-\nmijn parsoon naer Hollandt sal\nHeer van Amerongen\n\nMogen gaen en mijn Vaderlandt\nen vrou en kinderen sien, waer naer\nverlange, maer om dat het ijerse\nwesen tamelijck wel kenne, en sijn\nMajesteit sonder vreemde troepes dit\nrijck niet meester werden sal, de\nijeren verharden, en werden dagelijcks\nvuijlder, de doot van Heeren Alua\nen Webbenum is al een groot verlos\nin ons landt, en den staet verliest\ntwee goede officieren. nu geeft het\nBelegeren van Mons in Hollandt\neen grooten alarm en schrijft\nMen mijn dat den koning in persoon\nhet ontset tenteren wil. Godt geve\ndat het geluckt sij want hier aen\nhangt het gansche wesen. Van hier\nvalt voor dit mael weijnich niuws\nte schrijven, onder d'ijeren is een\ngerucht dat de franse generaels\nte Gallouaij sijn aengekomen en dat\nhet vier souden sijn St Rut, Tesse\nChammellij en Bosloo doch ick hebbe\ndaer noch geen sekerheijt van den\nLt Coll(onel) Eppinger sal ick seer gaern\nte dienste sijn, en blijve etc.\n\ngisteren is mijn eenen secretaris Bire\ngenaempt, een hups nuchteren jonck\nkarel, en soon van mijn Chirugijn\nop de straet staende spreecken tegens\nCornelle die int venster lach achter-\novergeslaegen, en de hanenpan\ningevallen waer aen hij dese mor-\ngen gestorven is, dit is een deerlh,?\nongeluck, Godt bewaere een ijder\nvoor diergelijcke.\nrec. a 30\nMolingaer den 22e feb 1691\nUEd.: missives vande 14 en 18 feb. sijn mijn\ngisteren behandicht, de vrienden maecken\nsich in Haeg vrolijck, en wij leggen hier\ninden dreck, soo gaen des werelts saecken\ndoch wij moeten den Almanack slachten\nen hoopen hier naer een beter, ick ben\nvoor twee daegen hier gekomen om\nde Vijant die met voetvolck Ruijterije\nen dragonders tussen Athloon, en dese\nplaets gekomen was, weder over de Shan-\nnon te drijven, maer wij hebben soo\nhoriblen quader weder van storm\nsneeuw, en regen dat wij niet avanceren\nkunnen, en ick beduchte dit desseijn\n\nvruchteloos maecken sal voor dit-\nmael heb geen tijdt meer waerom\nblijve etc\n\n\n\n\n\n\nrec: 1e Junij\nDublijn den 7/17  Maij 1691\nMet de laeste post hebbe ick geene\nbrieven van U.H.Ed, doch een seer lan-\nge van koning gekregen waer van alleen\nhet Hooft, en eijnde hier nevens gaet\nom U.H.Ed. te doen sien hoe goedertierent\nsijn Majesteit aen mijn schrijft, ick hebbe\nnu t'eenemael geresolveert de campag-\nne op s'konings begeren hier te doen\neen onverdrietelijcken Arbeijdt, lijf\nen leven voor deselve te waegen. Godt\nhoope ick sal mijn Majesteit waepenen\nzegenen en ons met victorie repa-\nrieren laeten, ick hebbe het geluck\n\ndat alle de generaels een vertrou-\nwen in mijn hebben, sijn Majesteit ver-\nandert die drie dat mijn leet doet\nte weeten Heer Gen: Douglas, Majesteit gene-\nraells Lanier en Kerk, dat mijn leet\ndoet, en die ick hadde gewenst te hou-\nden, en in haere plaets sullen komen\nMacquaij, Talmouts, en de Marquies\nde Ruvingnij, de voorige waeren mijne\nvrienden nu sal ick dese daer oock toe\nmaecken moeten, s'Gravemour verwach-\nte ick alle daeg en Willem Meester\nbeneffens soo danige ingenieurs als\nick vande koning geeijst hebbe, Het\nmeerendeels van onse Artellerije\nis gearriveert, en dagelijcks komen\nde Recruijtes, en kleeding over\nde Coll Goor een admirabel officier\nis besich met sijne Artellerie, en sal\ndese weeck op weijnige paerden nae\nklaer sijn, ick moet noch bidden dat U.H.Ed:\ndoch alle bedenckelijcke deviren ? aen wil\ndoen voorde deense Recruijtes, U.H.Ed.heeft\nnu met parmissie een dubbelt intrest\nhet gemeijn en een succes aen mijne\nonderneming, en ick hebbe het geluck\nmet den Hartoch van Wirtemberch en\nalle de deense generaells soo wel\n\nT'accorderen, dat wij noch noijt het\nminste different hebben gehad\nen met evengrooten ijver wenschen\ndat ons Meesters intrest te samen\nmach gaen daer toe Godt sijn segen\ngeven wl, die ick bidde voorde conte-\nnuatie van U.H.Ed. gesontheijt, en\nblijve etc\nick bidde dat desen brief aen niemant\nmach getoont werden\nHier nevens gaet oock het rapport mijn\ngisteren van eene seer fraije actie bij\neen van onse partijen gedaen\nrec. a 29e junij\nMolingaer den 2/12 junij 1691\nIck ben hier besich om des Konincks leger-\nte versaemelen, en hoop dat wij in korte\ndaegen sullen marcheren kunnen, de\nvijandt trekt sich oock te samen en sullen\nsoo geseijdt werdt wel dartich duijsent man\nsterck sijn, den Hartoch van Wirtenberch\nis in marsch derwaerts aen ick moet veele\nGuarnisoenen beset laeten, en moecke ? staet\ndat ons leger ontrent de 20000 man sal\nsterck sijn onse Artellerije is in goede\nstaet, en ons de noch over de Shannon\nmaecken moeten, het passeren van die Revier\nsal anders soo gemackelijck niet toe gaen\n\nEn daer mocht wel meerder bloedt dan\naende Boijn gestordt werden nu d'ijeren het\nfranse secours gekregen hebben sullen se in\nbeterstaet sijn als voorleden jaren alle de\nConfaerdije scheepen die haere provisen\nhebben gebracht waeren deense en sweetse\nscheepen, en hadden alle sulcke vlaggen\nop, dit is geen vrienden dienst, en men\nseijdt dat se in Vranckrijck over de drie\nmaenden lanck sijn ingehuurt geweest.\nMijn soon leijdt te Kilkennij sieck\naende landt sieckte, ick hebbe Gossteijn\nbij hem gesonden met mijn Chirugijn\ngeneraell, die voorsichtich en een goedt\nChirugijn en doctoor is, ick hoop\ndat hij haest sal beter werden, en\ndat se seijn H.Ed sullen kunnen naer\nDublijn brengen, ick blijve etc grooten\nhaest etc\n\nrecu.a. 9e julij\nInt leger bij Ballimoor\nden 13/23 junij 1691\nDen nevengaende brief is mijn gisteren\ndoor den Hartoch van Wirtenberch, toe\ngesonden met versoek dat se onder\nU.H.Ed. couvert aende Koninck mach wer-\nden behandicht, gedachte Heer Har-\ntoch staet mijn morgen te joinne-\nren en wachten wij alleen naer\nde blicke ponten met willem mees-\nter om de Rivier de Shannon te pas-\nseren aen welke passasie veel\ndependeren wil Godt geve ons geluck\nmijn soon betert Godt danck\n\nHandt, over handt, en sal haest\nweder harstelt sijn hij is een\ngrooten dans onsprongen want\nheeft doot kranck geweest, Het\nfranse secours is soo groot niet\nalse geroepen hebben, en Monsier\nde St. Rut houdt voorsichtelt-\nsijne transport scheepen bij hem\nof den hondt beet, dat sich retire-\nren konde waeckop is t'Athloon\nen doet mijn groote complimenten\nmaecken is seer t'onvreden\ndat wij Ballimoor soo haest\nhebben wech genomen ???-\nne officieren ophangen Het doet\nmijn leet dat aent deense Hof soo\nquade instrumenten sijn, die alles soo\nalles soo verkeert overbrengen\ndit sal niet laeten aende Hartoch\nvan Wirtenberch een Trou dienaer\nvan sijn koning te blijven soo\nwel als mijn, die alles sal contribue\nren wat mogelijck is, tot dienst vande\nDeense troupes, youl is geluckich\ndoot te sijn, ick hadde order om\n\nHem te casseren over sijne Vilaij-\nne schraperijen die hij in sijn winter\nquartier hadde gedaen het was\neen groote debauchant die alles ver-\nhoerde en verspeelde, en soo horribel\nvloeckte dat sijns gelijcken niet\nwas et ut vixit ita perixit\nhebbende alle het gelt van sijn regi-\nment verspeelt, en uit laetste\nmoment van sijn leven niet roepen\nals om de duijvel la forest most\nhet Regiment hebben, om dat hij\nd'eenichste generaell in Engelsen\nDienst was sonder Regiment, en\nhem van koning al voor lange\nversproocken was Monsieur duMon\nsal bij d'eerste occasie van harten\ntrachten dienst te doen en helpen\nsoo haest ick kan maer die vrien\nden blijven soo lang uijt dat niet\nfraij is, en die in militairen\ndienst eene goede occasie laet\nvoor bij gaen verliestse voor al\nsijn leven ick blijve etc\n\nrec 18e/28.\nAthloon den 1/11e julij 1691\nvan Heteren sal U.H.Ed. senden Copie\nvan't gepasseerde gisteren int inne-\nmen van Athloon, ick sal met drie\nwoorden daer bij voegen, dat er\ngeen troepen inde werelt braever\nkunnen sijn dan de deenen, en\ndat den Hartoch en Monsieur de Tet\nsou sich extraordinaris hebben ge-\nsingnaleert, wij sijn al v haer\nnaest Godt dese Victorie schuldich\nick blijve etc\n\nrec a 6e julij\nInt leger te Ballimoor den 9/19 junij 1691\nvolgens het schrijven vande Ritmeester\nGolsteijn is mijn soon Godt Lofs veel\nbeter en stondt hij als gisteren in\neen litiere naer dublijn gebracht\nte werden kunnen het bijden noch\nniet verdraegen soo dat ick hoop\nhij door sijn swaere sieckte is vrij\nsijn hier met een gedeelte vant\nleger t'sedert drie daegen voor\nde fortresse van Ballimoor welken\nplaets sich gisteren opgegeven\nHeer van Amerongen\n\nHeeft, en het Guarnisoen bestaende\nuijt t'sestich officieren, achthondert\ngesonde Soldaeten en twee hondert\nvijftich Rapparies, hebbe ick desen\ndach gevanckelijck naer dublijn\ngesonden wij hebben dese vrienden\ngoedt koop, het is eene stercke\nplaets en wij hebben niet meer\ndan ses dooden, en drie gequetsten\ndaer voor gehad, maer Goor heeft\nse met sijn Canon en Bomben\noverduijvelt, is een Admirabel\nman, en sal met Godts hulp grooten\ndienst doen de Konink sendt mijn\noock twee seer goede ingenieurs, nu\nsal haeste de groote swaerich aenkomen\nvande Shannon te passeren dat de meeste\ndifficulteijt wesen wil om dat de\nvijandt veel stercker dan wij sijn en\nsoo veele Rapparies voor haer hebben\nIck met permissie desen eijndigen\nen blijven etc\n\nRec. a  20e julij\nInt leger voor Athloon den 30 junij/10 jul: 1691\nMet de post van gisteren hebbe ick\nontfangen U.H.Ed.: missive vande 9/19 junij\nen verblijde mijn de continuatie van U.H.Ed.\ngesontheijt te vernemen, Het doet mijn\nleet dat de negotiatie van U.H.Ed. niet\nmeer avanceert, doch hier in moet men\npatientie hebben, Uijt mijn voorige\nbrieven heeft U.H.Ed: gelieven te sien\nHet veroveren van Ballimoor, en\nhet grootst gedeelte van Athloon, voor\nsoo veel alser van die stadt aen dese\nzijde de Shannon leijdt, de rest hebben\nwij door ons Canon en Bomben seer ge-\nruineert en tot een puijnhoop geschooten\nHeer van Amerongen\n\nsoo datter niets dan het Guarnisoen\nin duren kan die alle daegen uijt s'Vijans\nleger, t'welk achter de stadt staet,\nwerden af gelost, Ruijterije Gisteren meijnden\nwij door een guee vande Revier en over\neen Brugge te stormen doch soo als\nmen beginnen soude quam den Braadt\ndoor ongeluck inde fachienen op de Brugge\nhet welcke ons het desseijn heeft doen\nuijt stellen, aent veroveren van dese\nstadt, en het passeren vande Shannon\nwil veel gelegen sijn. Het leger vande\nvijant is tussen de 22000 en 30000 man\nsterck, soo d'overloopers seggen\nlijden te groot gebreck aen Broot-\nen krijgen somwijlen alle 24 somwijlen\nalle 9 uren maer een pint meel, en\ndit uijt oorsaeck dat se noch komen\nnoch paerden hebben om haer provsie\nnaer te voeren. Men seijdt datter veele\noverkomen sullen soo haest wij aen d'an-\nder sijde vande Shannon sijn geposteert, dat\nde tijdt moet leeren, ick doe wat kan\nen sal in s'konings dienst niets versuijmen\nGodt geve dat wij met geluck en voorspoet-\ndese campagne eijndigen mogen, daer voor\nde rust van Europa en den dienst secu-\nriteijt van onse koning niet weijnich\n\nAengelegen is, Het is mijn lief dat\nhet rapport vande secretaris Neven tot\navantasie vande Hartoch van Wirtenberg\nen mijn is sonder de waerheijt te spoe-\nren, en kander geen ander gedaen werden\nvoort overige moet ick patientie nemen\nen sal niet laeten te doen al wat in\nmijn macht is, tot dienst vande Deense\ntroepen die het buijten het diep respect\nsoo ick voor die koning hebbe meriteren\nen brave Soldaeten sijn waer van voor\ntwee daegen noch eene proeve hebbe\ngesien van drie deense Soldaeten die\nop den Hellen dach des smorgens\nWaepenen segenen het is een\nlust te sien, hoe de generaels\nofficieren, en Soldaeten ijder om\nseert niet alleen wel sijn gein-\nclineert, maer even ijverich om\nmaer aende vijant te komen\nen met als te grooten ijver son-\nde ons kunnen slaen wij sullen\nmet Godt de Rerier passeren\nen eene Bataille waegen, soo-\nde vijant staen wil, al waeren\nse veel stercker dan wij\n\nRec14 julij\nint leger voor Athloon\nden 21/31 Juny 1691\nIck hebbe d'eere gehad gisteren\neen langen brief van U.H.Ed: t'ont-\nfangen doch onmogelijck geen\ntijdt t'antwoorden. Wij sijn\nhier voor Athloon, hebben giste\nren de stadt aen dese sijde van\nRevier geattacqueert, en met\nweijnich verlos van volck in geno-\nmen, nu sullen wij van d'ander\nsijde attacqueren en ick hoop\nGodt sal verder des koninge\n\nMijn soon is Godt danck weder wel,\nen op wech om bij mijn te komen Ick\nblijve etc met grooten haest\nRec 3e aug\nAthloon den 5/15 julij 1691\nU.H.Ed. heeft uijt mijn voorige gelieven\nte sien, hoe geluckelijcken des konings\nArmee de stadt van Athloon vermees-\ntert heeft en ick geloof niet dat wij\ninde gansche Belegering en de twee stor-\nmen 60 dooden hebben, en sal de Vijant\nin tegendeel wel 2000 dooden hebben be-\nhalven de gevangens die over de 100 hon-\ndert sijn en waer onder is den generaell\nMajoor Maxfielt, die heden naer dublijn\nsende daeren boven soo heeft de Vijant\ndoot een Brigadier drie a vier Coll(onel)s\nHeer van Amerongen\n\nVerscheijde Lt Colle en Majoors behalven\neene groote quantiteijt mindere officie\nren se sijn in grooten haest geretireert\nen hebben twee stucken Canon in\nhet mouras wech gesmeeten, beneffens\neen mortier, die wij wel vinden sullen\nse hebben oock bereets Roscommon\nLainsbourough en Slego verlaeten\nde laeste plaets verbrandt en het\nCanon inde zee geworpen. Het is Godt\nden Heere die ons dese geluckige rencon-\ntre gegeven heeft, want wij soo-\nconsiderablen Revier als de Shannon-\nis gepasseert, hebben de Stadt\nstormender handt in genomen en\ndit alles int gesicht van s'Vijants le\nger dat stercker was dan t'onse, se\nbeginnen nu braef te deserteren\nen daer komender dagelijcks veele\nover. Men seijdt dat St. Rut eenige\ndames tracteerde in tijdt dat wij at-\ntacqueerden altoos de generaell\nMajoor Manfielt is van hem noch van\nL(uitenan)t gen(erael): duson die inde stadt met hem\nwesen moste niet voldaen. Wij mae-\nken staet morgen of overmorgen te\n\nMarcheren en verder op den Vijant\ntoe te gaen. Godt geve ons meerder\ngeluck, ick ben van harten verblijdt\ndat U.H.Ed. in soo grooten hoop is van\nsijne Tractaeten te voltrecken\nen blijve etc\nDen Hartoch van Wirtenberch\nis Godt Lof fris en gesont-\nrec 11 Aug\nInt leger te Aghrim den 19  julij 1691\nde groote victorie soo wij gisteren\nop de Vijant bevochten hebben\nsal U.H.Ed. sien uijt het relaes aen\nvan Heteren gesonden waertoe om\nde kortheijt des tijds mijn refneven\nmoet, ick sal alleen seggen dat\nde deenen ongemeijn gevochten heb-\nben en dat den Hartoch van\nWirtenberch geen kleijne deel-\naen dese victorie heeft, ljdt\n\nSij, daer voor gedanckt die de glorie?\ntoe kompt, en ickblijve etc\n\nrec. a 17e aug 169\nInt leger te Agrim den 15e julij 1691\nIck gaf mijn voor twee daegen d'eere\nU.H.Ed. inhaest te berichten de groote\nvictorie soo wij op den Vijant bevochten\nhebben, se bevindt sich inderdaet\nconsiderabelder, als wij ons hadden\nkunnen inbeelden, en bij aldien de\ndonckerheijt vande nacht ons niet had-\nde gestuijt daer soude gansch weijnich\nafgekomen sijn, den Coll: Goor sendt\nUwelGeb: eene relatie die soo correct\nis dat se niet kan verbetert weerden\nDe Vijant was over de 20000 man\nHeer van Amerongen\n\nte voet, en bij de 1000 paerden, alle\nwel gearmeerde, en gemonteerde troe-\npes starck, en wij waeren niet boven\nde 12000 man te voet, en tussen de 5 a\n6000 paerden, de Vijant was avantagesieus\ngeposteert, op een berch, die voor\ntwee darde parten van een mouras\ngedeckt was, waer over met moijte\nons voetvolck passeren konden, en\nonse Ruijterije most door een nauwe\npas twee a twee defileren. De Vijant\nsach ons van des smorgens om ses\nuren aenkomen en hadde den ganschen\ndach tijdt om sich in Bataille te\nstellen en wij konden niet voor\ndes avonts om vijf uren bij haer sijn\nwaerom men seer debatteerde, of\nment niet soude tot des anderen\ndaechs smorgens uijt stellen, maer\nomen Heer den Hartoch van Wirtenberch\nmijn d'eere doende van altoos met mijn\nte sijn, soo saegen wij een moevement\nvande Vijant waer door ons eenige con-\nfusie toe scheen, het welcke ons\naenstonts dede resolveren te beginnen.\nWij poucheerden lings en rechs af,\nmoedigden officieren en Soldaeten\naen, des vijans Canon dede ons in den\n\nBegin groote schade, en des Vijants in-\nfanterije vocht soo opiniater, als ick\nnoijt meer hebbe gesien, de piecken\nraeckten in malkanderen de musquet-\nten wierden om gekeert, en ick be-\nken dat een uur hebbe uijt gestaen\ndat noijt in grooter benouwtheijt\nben geweest, in sonderheijt wanneer\nick drie a vier van onse Bataillons\nopt mouras sach een weijnich achter\nuijt deijnsen die aenstonts door onse\nEngelse Ruijterije, alsoo die de rechter\nvleugel hadden en een voor een door\neen nauwe pas passeren mosten\nMet den degen inde handt wierden\ngesouteneert, en aenstonts weder\nharstelt, drie van onse franse Batail-\nlons waeren oock seer int nauw\nen door ses a seven vande Vijant geat-,\ntacqueert, ick vondt mijn juijst\nniet verre van haer, en een wech\nwaerdoor Ruijterije konde doen\navanceren tot groot geluck\nstonden drie Esquadrons deenen door\nden oversten donep gecommandeert\nniet verre van daer, die avanceeren\ndede, en met soo veel courasie\n\nen vigeur, in Vijant in braecken\nals men oijt verwachten konde.\nDe deense Guardes hadden met\nmeer als met vier Bataillons vande\nVijant te doen en pouceerden se\nalle vier, het grootste gevecht\nvande Ruijterije was op de lincker\nvleugel waer het Mouras geeijn\ndicht was, doch echter het landt\nal vrij gebroocken door hooge\nwallen, en het is remarkabel\ndatter niet een Esquadron noch\nBataillon van in gansche ar-\nmee is, dat niet gevochten, en\nExtraordinaris wel gevochten heeft.\nMijn soon heeft een Esquadron\nen chef gecommandeert, en sich\nBraef door de Vijants infanterije\ndoor geslaegen en een Bataillon\nover hoop gesmeeten, Ick hoop\ndat hij met Godt een goedt Soldaet\nwerden sal, wanneer hij noch meer\nsulcke occasien bij woonen kan.\nMen heeft mijn al acht en twi-\ntich vaendels en tien standaeren\nmet een paer keteltrommen ge-\nbracht behalven veele vaendels\n\nen standaeren, die de Soldaeten inde\nhadden in stuck gescheurt en dassen\nvan gemaeckt, de gevangens sijn\ntwee generaells Majoors Hamilton\nen dorrington vier a Vijf considera-\nble Mijlordts veele Coll.s Lts. Coll.s\nen Majoors, en wel vijf a ses hondert\ngemeijne soldaeten, Capitaijns en\ndiergelijcken. ick sendt se alle heden\nnaer dublijn, en de vaendels en\nstandaeren naer Engelandt aende\nConingin. Wij hebben tussen de\ndrie a vier hondert dooden waer\nonder de Gen. Majoor Holtsappell\n3\nis, en ontrent 700 gequetsten.\nvande Vijant sijn wel tussen de\n4 a 5000 op de plaets doot gebleven\nen haer meeste infanterije gede-\npaneert, St. Rut is en blijft onder\nhaer verlooren, soo dat hij doot is\nof moste onder onse gevangens\nsijn, dat men hem niet kende.\nmen sal se vande morgen een voor\neen doen naer sien, en het waer-\nniet goedt dat hij in handen van\nonse franse gerefugeerde viel, want\nhij is den grooten parsecuteijn\nin Vranckrijck geweest, en\n\nHeeft sijne passie in dit landt-\noock niet bedwingen kunnen\nmaer den Graef van paulijn\nmet eenige franse Ruijters en\nSoldaeten naer Vranckrijck gesonden\nom apparent oock qualijck getrac-\nteert te werden, ick hebbe hem\nseven a acht daegen geleden doen\nweeten dat soo hij gedrachte\ngraef, niet promptelt restituer-\nde, ick alle de franse gevangens\ndie ick hadde soude doen ophangen\nen waer onder verscheijde lt coll's\nMajoors en Capitains sijn en\nHeden sende ick het selfde com-\npliment aende Hartoch van Terconnell\nen dat mijn selver op d'ijerse\nMijlords vangeren sal soo hij mijn\ngedachte graef niet toe sendt\nwij hebben veele van des Vijants\nBagasie, en ses franse veltstucken\nverovert, en gisteren heeft\nsich het fort Bannachaer aen\nmijn over gegeven nu sullen wij\nnoch op Limmerick of Gallouaij\nmoeten los gaen en ick hoop dat\nGodt ons verder segenen sal ge-\nluck geven, en dat wij in korten\n\nDit Rijck onder des Konings gehoor-\nsaemheijt brengen sullen ick kan-\nniet genoch roemen de courasie\nen het beleijdt van alle de generaels\nin sonderheijt den Hartoch van\nwirtenberch die een Heere Sonder\nweergae is, en mach wel van\ngeluck spreecken dat voort eerste\nmaell van mijn leven, dat het\ngeluck hebbe van soo considerablen\ncommandement, soo braeve en\neerlijcke lieden hebbe aengetroffen\nde welcke mijn assisteren ja tot\nde minste Ruijter en Soldaet\ndaer kan niet braevers noch ij-\nveriggers tot s'konings dienst sijn\nen hoe kan ick Godt genoch dancken\ndat onder haer soo seer bea bemint\nmen ja int hevichst vant ge-\nvecht vlogen mijn Ruijters en sol-\ndaeten om den hals en embrasseer-\nden mijn, ick bidde dat U.H.Ed: dit\nmenageren wil, ick vinde mijn\ngeobligeert het aen hem te seggen\ndoch aen geen andere en weet Godt\ndat daer in geen glorie stelle maer\nse kompt hem alleen, die vande\nMenschen harten disponeert\n\nick bidde excuses dat aen Monsieur\nde falisau noch niet geantwoort\nhebbe salt per naesten doen en\nblijven etc\n\nrec.  a 17e augustus\nInt leger te Gallouaij den 22 Julij ouden Stijl 1691\nMet twee worden sal ick mijn deere\ngeven U.H.Ed: te seggen dat de stadt\nvan Gallouaij capituleert en aen\nstaende sondach in mijn handen wesen\nsal, dit is de sleutell van dit Rijck\nen Ick hoop dat Limmerick nu haest\nsal volgen, Monsieur de St Rut is\nen blijft doot, en d'andere geven\nmijn seer goede woorden, ick hoor\ndatse mijn haest een paspoort\neijschen sullen om naer Vranck-\nrijck te mogen gaen, se hadden\n\nGransch andere staet gemaeckt doch\nGodt heeft daer in voorsien\nIck blijve met haest etc\n\nRec. a: 7 september\nInt leger te Lougrea den 30e julij 10 augus 1691\nIck hadde gisteren voor middach hier met\nsijn Majesteit leger arriverende de eere van t'ont-\nfangen U.H.Ed: missive vande Rendesburch den 9\ngeschreven, die mijn vernemen doet dat U.H.Ed\nmet het deense Hof in Holsteijn aengekomen\nis. Godt geve U.H.Ed gesontheijt op sijne\nreijse, en dat hij die met contentement\nvoltrecken mach, Naer dat wij voorleden\nsondach vande stadt Gallouaij haden possissie\ngenomen, opde fortificatien Regering vande\nstadt, en het quarnisoen de noodige or-\ndres hadden gestelt soo ben gisteren\nHr. van Amerongen\n\nAls boven vermelt hier aengekomen\nmet intentie om verder naer Limmerick\nte marcheren, alwaer des vijants leger\nsich weder soo veel versamelt als moge-\nlijck is, ick geloof dat se moijte hebben\nsullen om een groot corps te samen\nte brengen, want haer voet volck soo\nofficieren als soldaeten deserteen seer\nen willen alle dienst hebben daer\nick met verlegen ben Baldarch odonel\neen ijersch Edelman die in Spaenjen, is\ngroot gemaeckt en waer van een oude\nijerse profetie seer veel van heeft\ngesproocken, als of bij d'eijren verlossen\nsoude, waer op dit bij geloovich volck\nveel staet hadden gemaeckt, heeft inde Ber-\ngen van't Graefschap Maijo, 3000 ijeren te\nsamen en gisteren aen mijn gesonden om\nte capituleren, doch sijne propositien\nsijn op sijn Spaensch extravagant, Bij\nde laeste Bataille heeft den ijersen Adel\nseer aengeloopen, en wij hebben geexpe-\nrimenteert dat daer voor Goedt, en Relisie\ngevochten werdt, het gevechtscharp valt\nMonsieur du Son france Lt generaell\nwas in Gallouaij trock met 3000 Man-\nuijt, en quam des smorgens mijn aen\nmijn tent een compliment maecken\nde viconte de dillon ijerse gouverneur\n\nvan die plaets dede ick des smiddachs\nmet mijn eeten en gelijck hij Schoonsoon\ndan Terconnell is, soo seijde hem dat\nhet noch tijdt was, de waepenen neder\nte leggen, en raisonnable conditien t'Am-\nbrasseren, waer toe vande Koninck mijn\nMeester ordres hebbe, ick hoor dat ond-\nte Limmerick hier over Krijsraet gehou-\nden is, doch geconcludeert se souden noch\neerst een courier naer Vranckrijck senden\nondertussen soo hebbe ick acht a tien\nEngelse en Hollantse fregatten die\nop mijn vertreck van Gallouaij voor die\nstadt quamen geordonneert, inde Schan-\nnon te seijlens om de correspondentie\nBij zee voor Limmerick te beletten, en\nIck hoop binnen een dach tien a twaelf\nmet d'Armee daer voor te staen. Het re-\ngent hier dagelijcks dat de Wegen seer\nquaedt maeckt en mijn groote moijte\nvoor mijne Artellerije geeft, ja verre\ndoet om marcheren. mijn soon praesenteert\nsijn schuldigen dienst en ick blijve etc.\n\nrec a 10e sept\nTerhoch den 7/17 Augsutus 1691\ntwee posten sijn desen dach uijt Hollandt\ngearriveert doch brengen mijn geene brieven\nvan U.H.Ed. die de Vrou van Amerongen\nmijn schrijft dat te Hamburch was geweest\nen weder naer denemarcken vertrocken\nGodt geve dat dese reijse U.H.Ed. wel\nbekomen sij, en dat hij noch lange jaeren\ndiergelijcke met gesontheijt mach doen\nIck ben gisteren met des konings Armee\nweder hier ontrent twintich mijlen\nvan Limmerick gearriveert, hebbe\nHeer van Amerongen\n\nDit detour om de subsistentie en quade\nwegen doen moeten, de vijant staet on-\nder het Canon van Limmerick, en verster-\nken haer met alderhande Rapparies\nse seggen noch eens te willen vechten\ndat de tijdt leeren sal Monsieur du Zon\nfranse lt generaell commandeert nu\nhaer Armee hij was in Gallouaij quam\nbijt uijt trecken van't Guarnisoen mijn\naen mijn Tent een beleeft compliment\nmaecken, ick regaleerde hem met\neen dozijn boutelles Canay seck\nHij sal mijn apparent uijt Limme-\nrick, soo veel Canon Cogels inde plaets\nte rugge senden, den Brigadier Ellen-\nberch en Monsieur dumon sijn eijndelijck\ngearriveert, den Hartoch van Wirten-\nberch is niet wel voldaen over haer\nlang uijtblijven, waer in hij groote\nreden heeft, want achterstallen\nt'ontfangen, noch qualijck te spree-\nken, en het merendeels vande Campagne\nuijt te blijven past geen goedt officier\nU.H.Ed. kan niet dencken hoe braef-\n\ngenereus, eerlijck en oprecht, den\nHartoch van Wirtenberch is, Godt gave\ndat alle soo waeren en men soude haest\neen groote verrandering ten besten van\nEuropa sien nu Godt wil se beheeren\ndiet niet en sijn, en wij sullenden\nons best voor vechten, ick bidde\nGodt voor U.H.Ed. lang leven en blijve\netc\nde koning heeft mijn bij een brief\nvan sijn eijgen handt betuijgt verge-\nnoecht te sijn over mijne contentie\nin bevochtene victorie\nrec a 2 september\nint leger te Carkenlisch den 17/27 August  1691\nWij sijn met des Konings leger voor twee\ndaegen hier vijf Mijlen van Limmerick gearr-\nveert alwaer onse groote artellerije sijn\nwachtende om dan de stadt te naederen\nGodt hoope ick sal aen des koning waepenen\nhet selfde succes geven, dat wij tot noch toe-\nhebben gehad, en dat dit koninckrijck haes-\nten eenenmaell sal wesen onder gebracht\nDen Hartoch van Wirtenberch en de deen-\nsche troepes hebben geen weijnich part\nen deel aen alle dese victorien en ick\n\nBen geluckich met soo braeven generaell\nen officieren te dienen, ick verblijde mijn\ndat U.H.Ed. weder inde gesontheijt te Coppen-\nhagen is gearriveert Godt laete het lang\nduren en geve dat ick U.H.Ed: haest int\nvaderlandt opwachten mach, inmiddels\nsoo blijve etc mijn soon presentaert\nsijn gehoorsamen dienst,\n\nrec a 2 october\nint leger voor Limmerck den, 1/11sept 1691\nT'sedert mijnen laesten hebben wij geene\nbrieven uijt Hollandt noch Engelandt\ngehad en sijn hier noch besich met\nde stadt van Limmerick soo veel moge\nlijck te benauwen, en hebben begost\nbomben en Carcassen inde stadt te wer-\npen het Guarnisoen is seer starck\nen se sijn gesonteneert van alle haer\nRuijterije die over de Rivier staet\nsoo dat het difficiell vallen wil haer\ndoor ordinaire Approches het nemen\nvane haere buijten wercken en\nHeer van Amerongen\n\nopenen van een bres t'attacqueren\nmen sal se echter door d'een of ander\nmannier soo danich trachten te ruin-\nren dat se beswaerlijck de Winter substi-\nteren sullen t'sedet de doot van\nColl. donep hebben wij niemant meer\nvan consideratie verlooren, dit is al\nhet geene voor dit maell van hier\nte schrijven is, ick blijve etc\n\nMonsieur\nMonsieur te Baron de\nReede signeur d'Ameron-\ngen, Ginckell Elst Chevallier\nde l'ordre de Denemarck etc\nrec. a 2e october\nInt leger voor Limmerick den 2/9 september 1691\nHet is heden den vierden dach dat wij voor\nLimmerick sijn welcke stadt sich schijnt\nte willen defenderen, of wel wij van\nde hoochtens sonder groote resistentie\nmeester geworden sijn, morgen sullen\nonse baterijen beginnen te speelen\nen ick ben geresolveert met een gedeelte\nvan ons leger over te gaen om s'vijants\nRuijterije aen d'ander sijde vande rivier\nist doenlijck t'attacqueren, en al soo\n\nde stadt van weder sijde te sluijten\nmiddeler wijl dat den Hartoch van Wirten-\nberch se van dese sijde attacqueert, en\nsijn wij vast besich het geene vant\nleger hier blijven sal te retrancheren\nen secuur te setten, de IJeren blijven\nopiniater, en het heeft geen kleijne\ndifficulteijt een stadt te vermeesteren\ndaer een leger in staet en die wel is\ngefortificeert, doch wij sullen ons\nbest doen, en het werck met godes\nhulp uijt voeren soot doenlijck\nis, ick bidde vergiffenisse dat\ndese niet langer maecke de tijdt\nmancqueert mijn, ick blijve etc\nDen Coll donep is eergisteren doot\ngeschooten des snachs onder de stadt\npatroellerende\n\nBlijven, wil dese stadt noch menschen\nkosten want de fortificatien sijn veel\nverbetert, en daer is een leger in Morgen\nsal de swaere Artellerije bij ons sijn\nHet quade weder begint Godt danck te\nbedaeren Den Hartoch van Terconnell\nis overleden binnen Limmerick dit\nhadde bij mijnen laesten vergeten te\nschrijven, opde flatterijen van dijck\nvelt, kan niet anden seggen als dat het\nsijne eerste niet sijn, sijn Neef Wilsun\nhebben se voor weijnige daegen in Lim-\nmerick gevangen geset, op suspitie\n\nRec a . 2e october\nCarkenlisch den 24 Aug 4e september 1691\nIck hadde gisteren het geluck t'ontfangen\nU.H.Ed. missive vande 9/19 Augs, wij staen mor-\ngen op Limmerick toe te marcheren ick\nhoop Godt sal ons gelucken voorspoedt\ngeven, op dat de campagne met succes\neijndigen mach, en dan repatieren\nals wanneer hoope dat de koning een\nmaell sijn goedtheijt aen mijn en mijne\nfamilie toonen sal, en dat hij nu siet\ndat geen arbeijt spaere en daegelijcks\nmijn leven hasardere om sijn Majesteit\ndienst te doen soo de ijren opiniater\n\ndat hij aen mijn de stadt overleveren\nwilde, hij is tot sulcke Actien bequaem\nick moet met groeten haest eijndigen, en\nblijven etcs mijn soon presenteert, sijn\ngehoorsamen dienst\nrerc a 15e september\nint leger voor Limmerick den 11/21 sept 1691\nHet is nu een gemijnen tijdt dat ick het\ngeluck niet hebbe, brieven van U.H.Ed. t'ont-\nfangen, wij sijn voorleden dijnsdach veertien\ndaegen voorde stadt van Limmerick gekomen\ndie wij bombarderen en Cannonneren het Guar-\nnisoen is soo sterck het saisoen soo verne\nverloopen, en de stadt in veel beter defen-\ntie als het voorleden jaer al het welcke\nons heeft doen resolveren deselve met\ninde forme aen te tasten, maer alse sal\nwesen gebombardeert soo sal men trachten\nhaere Ruijterije te disperneren\nHeer van Amerongen\n\nEn haer soo veel mogelijck de verdere sub-\nsistentie te benemen op dat se sich eijnde\nlijck op geven het welcke onfeijlbaer vallen\nmoet soo Vranckrijck haer geen prompt\nsecours en sendt, het welcke ick hoop\ndat, door de vloot of een gedeelte van\ndien sal verhindert werden, de franse\nparsuasien doet d'eijeren noch opiniate\nblijven, en het schijnt se kunnen door\ngeen middell dan gewelt gedwongen wer-\nden, dese nacht hebben wij op twee\na drie plaetsen de stadt van Limme-\nrick in Brandt geschooten doch ick\nweet noch niet wat effect dit heeft\ngedaen, den Heere Hartoch van Wirtenberch\nis altoos fris presonteert U.H.Ed sijn dienst\nen drinckt somwijlen sijn gesondtheijt\nMonsieur de Man is voor drie daegen\ngearriveert, heeft mijn U.H.Ed. brief over\ngelevert, ick sal hem helpen soo haest\nals daer toe occasie hebbe, maer de conink\nheeft mijn verscheijdene vollontairen\ngesonden die de gansche campagne gedien\nhebben, en sullen moeten voorgaen Het\nis facieus dat de Campagne in Vlaenderen\neijndicht sonder dat ter iets considerabels\nheeft kunnen verricht werden en dat\n\nvan gelijcken onse groote vloot sich soo\nmoet retireren, ick blijve etc\n
"""

In [17]:
import pandas as pd
import json
import re
import os


class DataProcessor:
    """A class to load, process, and provide a filtered DataFrame with full text."""

    def __init__(self, csv_path, json_path):
        """Initializes the DataProcessor with file paths."""
        self.csv_path = csv_path
        self.json_path = json_path
        self._workable_df = None  # Initialize the private attribute

    def _clean_text(self, text):
        """Cleans text by removing newlines, extra whitespace, and fixing hyphens."""
        if not text:
            return None
        text = text.replace("\n", " ")
        text = re.sub(r'\s+', ' ', text).strip()
        text = re.sub(r'(\w+)-\s+', r'\1', text)
        return text

    def _load_and_process_data(self):
        """Loads, filters, and merges data from CSV and JSON files."""

        # Load and preprocess CSV
        try:
            df = pd.read_csv(self.csv_path)
            print("CSV Column Names:", df.columns.tolist())  # Print statement added
        except FileNotFoundError:
            print(f"Error: CSV file not found at {self.csv_path}")
            return None

        # Example renaming column if column is called "Choice"
        # df = df.rename(columns={"Choice": "Keuze"})
        # Correct code if column is called keuze
        df['Keuze'] = df['Keuze'].str.upper()
        df_filtered = df[df['Keuze'] == 'JA'].copy()

        # Load and preprocess JSON
        try:
            with open(self.json_path, "r", encoding="utf-8") as f:
                json_data = json.load(f)
        except FileNotFoundError:
            print(f"Error: JSON file not found at {self.json_path}")
            return None

        # Create dictionary from JSON data and clean text
        if isinstance(json_data, dict):
            json_dict = {key: self._clean_text(value) for key, value in json_data.items()}
        else:
            json_dict = {item['invnr']: self._clean_text(item['content']) for item in json_data}

        return df_filtered, json_dict


    def _get_full_text(self, invnr, json_dict):
      """Retrieves the full text from the JSON dictionary."""
      return json_dict.get(invnr, None)


    @property
    def workable_df(self):
        """Returns a DataFrame with full text available, creating it if it doesn't exist."""
        if self._workable_df is None:
            df_filtered, json_dict = self._load_and_process_data()
            if df_filtered is None:
                return None
            df_filtered.loc[:, "Full text"] = df_filtered["invnr"].apply(lambda invnr: self._get_full_text(invnr, json_dict))
            self._workable_df = df_filtered[df_filtered['Full text'].notna()]
        return self._workable_df

    def display_diagnostics(self):
        """Displays diagnostic information about the DataFrame."""
        if self.workable_df is None:
            print("Error: No workable DataFrame available. Check file paths.")
            return

def main():
    # Define file paths
    csv_path = "../data/prototyping/HUA-batch.csv"
    json_path = "../data/prototyping/HUA-fulltext.json"

    # Create DataProcessor instance
    data_processor = DataProcessor(csv_path, json_path)
    return data_processor



In [18]:
result = main()

In [19]:
docs = []
for _, doc in result.workable_df.iterrows():
    meta = doc[['ID', 'AET_ID', 'num_scans', 'invnr', 'beschrijving', 'link', 'representatieve afbeelding', 'Context en beschrijving (aan elkaar geplakt)']].to_dict()
    document = Document(content=doc['Full text'], meta=meta)
    docs.append(document)

CSV Column Names: ['Keuze', 'ID', 'AET_ID', 'num_scans', 'invnr', 'GUID', 'beschrijving', 'link', 'representatieve afbeelding', 'visueel of tekstueel', 'Soort / brontype', 'Bron subtype', 'Personen', 'Locaties', 'Periodes', 'Onderwerpen', 'Full text', 'Context', 'Context en beschrijving (aan elkaar geplakt)']


C:\Users\Niek\AppData\Local\Temp\ipykernel_33008\1597641408.py:71: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None
 None None None None None None None None None
 '1667 juni-aug Middachte den 18 ijuin 1667, Mijn heer en lieste hartge door dien de post in voorleedene dach de briefve so laet brocht heb ick die van uhEd vande deeser doen niet beantwoorde het welcke daerom nu mijn kleijn vermooge sal doen, en voor Eerst segge, dat ick tot mijn leetweese den heere roodeburch niet heb konne aentreffe, want so den Espresse die, ick afgesonde had aen wijburch on naer hem te ver

In [20]:
docs

[Document(id=5621dea2835992f968043ea2baa7460dcd3a55f597ba6f0a19df85fb2c54f1c4, content: '1667 juni-aug Middachte den 18 ijuin 1667, Mijn heer en lieste hartge door dien de post in voorleede...', meta: {'ID': 722001.0, 'AET_ID': 4.0, 'num_scans': '236', 'invnr': '1001.2722', 'beschrijving': 'Brieven gericht aan Godard Adriaan van Reede afkomstig van zijn echtgenote Margareta Turnor, 1667-1691 2722.  1667 juni-aug., 1671 sept.-1672 dec', 'link': 'https://hetutrechtsarchief.nl/collectie/609C5B9947B64642E0534701000A17FD', 'representatieve afbeelding': 'https://proxy.archieven.nl/large/39/609C5BCADA7D4642E0534701000A17FD', 'Context en beschrijving (aan elkaar geplakt)': '1001 Huis Amerongen  /  Inventaris / 2. Stukken betreffende heren en vrouwen van Amerongen en Zuilenstein en hun bloedverwanten / 2.1. Familie van Reede / 2.1.07. Godard Adriaan van Reede x Margareta Turnor / 2.1.07.1. Persoonlijk / Brieven gericht aan Godard Adriaan van Reede afkomstig van zijn echtgenote Margareta Turnor,

In [21]:
def get_claude_client() -> Anthropic:
    return Anthropic(api_key=os.getenv("CLAUDE_API_KEY"))

def translate(text: str) -> str:
    client = get_claude_client()
    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=2048,
        system="Je bent een expert in het vertalen van oud-nederlandse stukken tekst. Het is aan jou om deze tekst te vertalen naar duidelijk en modern Nederlands. Let erop dat sommige woorden met koppeltekens en newlines zijn geschreven. Reageer ENKEL met de vertaalde tekst.",
        messages=[
            {"role": "user", "content": f"Vertaal het volgende stuk oud-nederlands naar het modern Nederlands. Gebruik enkel logische, samenhangende zinnen. Geef ENKEL de vertaling terug: {text}"},
        ],
    )
    return response.content[0].text


@component
class DocumentTranslator:
    @component.output_types(docs=List[Document])
    def run(self, documents: List[Document]) -> Dict[str, List[Document]]:
        translated_documents = []
        with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
            futures = []
            for document in documents:
                current_text = document.content
                future = executor.submit(translate, current_text)
                futures.append((document, future))

            for document, future in tqdm(futures):
                translated = future.result()
                document.content = translated
                translated_documents.append(document)
        return {"docs": translated_documents}

In [22]:
def setup_processing_pipeline():
    pipeline = Pipeline()
    
    pipeline.add_component("cleaner", DocumentCleaner())
    pipeline.add_component("translator", DocumentTranslator())
    pipeline.add_component("splitter", DocumentSplitter(split_by="sentence", split_length=10))

    pipeline.connect("cleaner", "translator")
    pipeline.connect("translator.docs", "splitter")
    
    return pipeline

In [23]:
pipeline = setup_processing_pipeline()

result = pipeline.run(data={
    "cleaner": {"documents": [docs[1]]}},
    include_outputs_from=["cleaner", "splitter", "translator"]
)

INFO:haystack.core.pipeline.pipeline:Running component cleaner
INFO:haystack.core.pipeline.pipeline:Running component translator
  0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
100%|██████████| 1/1 [01:00<00:00, 60.85s/it]
INFO:haystack.core.pipeline.pipeline:Running component splitter


In [24]:
result.get("splitter")

{'documents': [Document(id=7e9938197aed9ac62579d795761e660838e829886a7a15dd0bc41bbf10c7b14b, content: 'Hier is een moderne Nederlandse vertaling van de tekst:
  
  Den Haag, 2 januari 1673
  Ontvangen 13 janua...', meta: {'ID': 722002.0, 'AET_ID': 4.0, 'num_scans': '187', 'invnr': '1001.2723', 'beschrijving': 'Brieven gericht aan Godard Adriaan van Reede afkomstig van zijn echtgenote Margareta Turnor, 1667-1691 2723.  1673 jan.-sept', 'link': 'https://hetutrechtsarchief.nl/collectie/609C5B9947B74642E0534701000A17FD', 'representatieve afbeelding': 'https://proxy.archieven.nl/large/39/609C5BCAE68F4642E0534701000A17FD', 'Context en beschrijving (aan elkaar geplakt)': '1001 Huis Amerongen  /  Inventaris / 2. Stukken betreffende heren en vrouwen van Amerongen en Zuilenstein en hun bloedverwanten / 2.1. Familie van Reede / 2.1.07. Godard Adriaan van Reede x Margareta Turnor / 2.1.07.1. Persoonlijk / Brieven gericht aan Godard Adriaan van Reede afkomstig van zijn echtgenote Margareta Turnor, 